# Get data from twitter

In [ ]:
!pip install git+https://github.com/Museum-Barberini/twint.git@origin/master#egg=twint

In [ ]:
!pip install nest_asyncio -q

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import twint
import sys
module = sys.modules["twint.storage.write"]

tweets = []


def Json(obj, config):
  tweet = obj.__dict__
  tweets.append(tweet)

module.Json = Json

def getRecentTweetsForGivenTopic(topic, limit=500, language='en'):
  global tweets
  
  config = twint.Config()
  config.Search = topic
  config.Limit = limit
  config.Output = "tweets.json"
  config.Store_json = True
  config.Hide_output = True
  config.Lang = language

  twint.run.Search(config)
  copied_tweets = tweets.copy()
  tweets = []
  print(topic,"Done")
  return copied_tweets

In [ ]:
ipl = getRecentTweetsForGivenTopic('IPL 2022')

In [ ]:
!pip install emoji -q

In [ ]:
import pandas as pd
import re
import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

In [ ]:
df = pd.read_csv("Sentiment Models Results on IPL 2022.csv")
ipl = df['Tweets'].values

In [ ]:
ipl

# Data cleaning

In [ ]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    # tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
    #      if w.lower() in words or not w.isalpha())
    return tweet

In [ ]:
tweet = ipl[7]
tweet, cleaner(tweet)

# Setting up models

In [ ]:
!pip install sentencepiece==0.1.95 -q
!pip install transformers==4.12.3 -q

     |████████████████████████████████| 1.2 MB 4.6 MB/s 


In [ ]:
%cd /content
!mkdir models
%cd models
!mkdir model_files
%cd model_files

!git lfs install
!git clone https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment

!git lfs install
!git clone https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis

%cd /content

/content
mkdir: cannot create directory ‘models’: File exists
/content/models
/content/models/model_files
Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'bert-base-multilingual-uncased-sentiment'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 39 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (39/39), done.
Filtering content: 100% (3/3), 1.87 GiB | 64.57 MiB/s, done.
Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'bertweet-base-sentiment-analysis'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 28 (delta 8), reused 24 (

In [ ]:
# %cd /content/models/model_files
# !git lfs install
# !git clone https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis

In [ ]:
%cd /content

/content


# Inference

In [ ]:
import torch
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline

from scipy.special import softmax

SENTIMENT_ANALYSIS_CONFIG = {
    "sentiment_model_path": "./models/model_files/bertweet-base-sentiment-analysis",
    "sentiment_mapping": {"0": "NEGATIVE", "1": "NEUTRAL", "2": "POSITIVE"}
}

SENTIMENT_ANALYSIS_5_CLASS_CONFIG = {
    "sentiment_model_path": "./models/model_files/bert-base-multilingual-uncased-sentiment",
}


class Sentiment:
    def __init__(self, config=SENTIMENT_ANALYSIS_CONFIG):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        print(f"Transformers version : {transformers.__version__}")
        print(f"Using device : {self.device}")

        model_dir = config["sentiment_model_path"]
        self.mapping = config["sentiment_mapping"]
        print(model_dir)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(self.device)
        self.model.eval()

        self.tokenizer = AutoTokenizer.from_pretrained(model_dir)
        print("Transformer model from path %s loaded successfully", model_dir)

    def inference(self, input_text):
        # print(f"Received text: {input_text}")
        input_text = input_text.encode().decode("utf-8")
        max_length = 128

        inputs = self.tokenizer.encode_plus(
            input_text,
            max_length=max_length,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_tensors="pt",
        )

        input_ids = inputs["input_ids"].to(self.device)
        attention_mask = inputs["attention_mask"].to(self.device)

        predictions = self.model(input_ids, attention_mask)

        prob_dict = {}

        out = predictions[0][0].unsqueeze(0)
        out = softmax(out.cpu().detach().numpy())[0]

        for i, prob in enumerate(out):
            prob_dict[self.mapping[str(i)]] = eval(str(prob))

        return max(prob_dict, key=prob_dict.get)


class Sentiment_5_Class:
    def __init__(self, config=SENTIMENT_ANALYSIS_5_CLASS_CONFIG):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        print(f"Transformers version : {transformers.__version__}")
        print(f"Using device : {self.device}")

        model_dir = config["sentiment_model_path"]
        print(model_dir)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(self.device)
        self.model.eval()

        self.tokenizer = AutoTokenizer.from_pretrained(model_dir)
        print("Transformer model from path %s loaded successfully", model_dir)

        self.classifier = pipeline('sentiment-analysis', model=self.model,
                                   tokenizer=self.tokenizer)

    def inference(self, input_text):
        # print(f"Received text: {input_text}")
        input_text = input_text.encode().decode("utf-8")
        max_length = 128

        inputs = self.tokenizer.encode_plus(
            input_text,
            max_length=max_length,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_tensors="pt",
        )

        input_ids = inputs["input_ids"].to(self.device)
        attention_mask = inputs["attention_mask"].to(self.device)

        predictions = self.model(input_ids, attention_mask)

        prob_dict = {}

        scores = predictions[0][0]
        scores = softmax(scores.cpu().detach().numpy())

        prob_dict['NEGATIVE'] = float(scores[0] + scores[1])
        prob_dict['NEUTRAL'] = float(scores[2])
        prob_dict['POSITIVE'] = float(scores[3] + scores[4])

        return max(prob_dict, key=prob_dict.get)


In [ ]:
sa1 = Sentiment()

Transformers version : 4.12.3
Using device : cuda
./models/model_files/bertweet-base-sentiment-analysis
Transformer model from path %s loaded successfully ./models/model_files/bertweet-base-sentiment-analysis


In [ ]:
sa2 = Sentiment_5_Class()

Transformers version : 4.12.3
Using device : cuda
./models/model_files/bert-base-multilingual-uncased-sentiment
Transformer model from path %s loaded successfully ./models/model_files/bert-base-multilingual-uncased-sentiment


In [ ]:
sa1.inference(ipl[2])

In [ ]:
sa1.inference("Congratulations Gujarat Titans!   The 2022 Indian Premier League final was played on 29 May 2022 at the Narendra Modi Stadium in Ahmedabad.  GT beat RR by 7 wickets in the final.  #gt #congratulations #gujarat #ipl #narendramodi #stadium #ahmedabad #t20 #win #winner #tournament  https://t.co/vlB4GGy3CV")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'POSITIVE'

In [ ]:
results = {}

In [ ]:
cleaned_tweets = [cleaner(tweet) for tweet in ipl]

In [ ]:
from time import time

In [ ]:
s = time()
df['BERTweet_clean'] = [sa1.inference(tweet) for tweet in cleaned_tweets]
f = time()

print("Time taken: ", f-s)

In [ ]:
s = time()
df['bert-base-multilingual-uncased-sentiment_clean'] = [sa2.inference(tweet) for tweet in cleaned_tweets]
f = time()

print("Time taken: ", f-s)

In [ ]:
df

In [ ]:
df.to_csv("results.csv",index=False)

## Roberta

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
# labels=[]
# mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
# with urllib.request.urlopen(mapping_link) as f:
#     html = f.read().decode('utf-8').split("\n")
#     csvreader = csv.reader(html, delimiter='\t')
# labels = [row[1] for row in csvreader if len(row) > 1]
labels = ['NEGATIVE', 'NEUTRAL', 'POSITIVE']

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [ ]:
labels

In [ ]:
def roberta_inference(text):
  print(f"Recieved text : {text}")
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)


  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  return labels[ranking[0]]

In [ ]:
from tqdm import tqdm

In [ ]:
s = time()
df['twitter-roberta'] = [roberta_inference(tweet) for tweet in ipl]
f = time()

print("Time taken: ", f-s)

In [ ]:
s = time()
df['twitter-roberta_clean'] = [roberta_inference(tweet) for tweet in cleaned_tweets]
f = time()

print("Time taken: ", f-s)

In [ ]:
df

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
def vader_inference(text):
  r = sid.polarity_scores(text)
  del r['compound']
  return max(r, key=r.get)

In [ ]:
df.to_csv("results.csv", index=False)

In [ ]:
s = time()
df['VADER'] = [vader_inference(tweet) for tweet in ipl]
f = time()

print("Time taken: ", f-s)

In [ ]:
s = time()
df['VADER_clean'] = [vader_inference(tweet) for tweet in cleaned_tweets]
f = time()

print("Time taken: ", f-s)

In [ ]:
df.to_csv("results.csv", index=False)

In [ ]:
df.to_excel("Sentiment models.xlsx", index=False)

In [ ]:
results['Tweets'] = [i['tweet'] for i in ipl ]

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(results).to_csv("Sentiment Models Results on IPL 2022.csv", index=False)

# [RETWEET](https://www.kaggle.com/datasets/soroosharasteh/retweet)
Dataset of Tweets and overall predominant sentiment of their Replies

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

In [ ]:
def get_tweet_from_id(id):
  print(id)
  url = f"https://publish.twitter.com/oembed?dnt=true&omit_script=true&url=https://mobile.twitter.com/i/status/{id}"
  try:
    r = requests.get(url)
    
    text = r.json()['html']
    tweet_url = r.json()['url']

    text = text.split("</p")[0]
    text = text.split("pic.twitter.com")[0]
    
    return text, tweet_url

  except Exception as e:
    print(id, e)
    return '', ''

In [ ]:
get_tweet_from_id(1163887022504906752)

In [ ]:
get_tweet_from_id(1179631197405642753)

In [ ]:
def clean_html(html_text):
  # CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  # cleantext = re.sub(CLEANR, '', html_text)
  # return cleantext
  cleantext = BeautifulSoup(html_text, "html.parser").text

  return cleantext

In [ ]:
clean_html(get_tweet_from_id(1544677961777152000)[0])

## Get training data

In [ ]:
import pandas as pd

In [ ]:
training_df = pd.read_csv("train_final_label.txt", delimiter='\t')
training_df

In [ ]:
training_df['id'].value_counts()

In [ ]:
clean_df = training_df.drop_duplicates('id')

In [ ]:
clean_df

In [ ]:
df1 = clean_df[:1000]

In [ ]:
clean_df['url'] = clean_df.id.apply(get_tweet_from_id).str[1]

In [ ]:
from tqdm import tqdm

In [ ]:
urls = []
html = []
cleaned_tweets = []

for id in tqdm(df1.id.values):
  tweet, url = get_tweet_from_id(id)
  clean_tweet = clean_html(tweet)

  html.append(tweet)
  urls.append(url)
  cleaned_tweets.append(clean_tweet)

In [ ]:
cleaned_tweets

In [ ]:
df1['url'] = urls
df1['html_tweet'] = html
df1['tweet'] = cleaned_tweets

In [ ]:
df1.to_csv("RETWEET Sample training data.csv", index=False)

# Training data

In [ ]:
!pip install fasttext emoji -q

In [ ]:
import fasttext
import sys
import os
import nltk
nltk.download('punkt')

import pandas as pd
import datetime

from bs4 import BeautifulSoup

import re
import itertools
import emoji

In [ ]:
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }


In [ ]:
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }


In [ ]:
def strip_accents(text):
    if 'ø' in text or  'Ø' in text:
        #Do nothing when finding ø 
        return text   
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def tweet_cleaning_for_sentiment_analysis(tweet):    
    
    #Escaping HTML characters
    tweet = BeautifulSoup(tweet).get_text()
    #Special case not handled previously.
    tweet = tweet.replace('\x92',"'")
    #Removal of hastags/account
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())
    #Removal of address
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    #Removal of Punctuation
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    #Lower case
    tweet = tweet.lower()
    #CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS = load_dict_contractions()
    tweet = tweet.replace("’","'")
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    # Standardizing words
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    #Deal with smileys
    #source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    #Deal with emojis
    tweet = emoji.demojize(tweet)
    #Strip accents
    tweet= strip_accents(tweet)
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())
    
    # DO NOT REMOVE STOP WORDS FOR SENTIMENT ANALYSIS - OR AT LEAST NOT NEGATIVE ONES

    return tweet

In [ ]:
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import numpy as np


from sklearn import model_selection, preprocessing, metrics, linear_model, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import BorderlineSMOTE, SMOTE, ADASYN, SMOTENC, RandomOverSampler
from imblearn.under_sampling import (RandomUnderSampler, 
                                    NearMiss, 
                                    InstanceHardnessThreshold,
                                    CondensedNearestNeighbour,
                                    EditedNearestNeighbours,
                                    RepeatedEditedNearestNeighbours,
                                    AllKNN,
                                    NeighbourhoodCleaningRule,
                                    OneSidedSelection,
                                    TomekLinks)
from imblearn.combine import SMOTEENN, SMOTETomek

In [ ]:
data = pd.read_csv("train.tsv", delimiter='\t')
data

In [ ]:
data.label.value_counts()

In [ ]:
data[data.label == -1]

In [ ]:
# data['label'] = data['label'].replace('positive',1)
# data['label'] = data['label'].replace('negative',-1)
# data['label'] = data['label'].replace('neutral',0)

In [ ]:
data

In [ ]:
data['cleaned_tweets'] = data.tweet.apply(tweet_cleaning_for_sentiment_analysis)

In [ ]:
from nltk.tokenize import TweetTokenizer
  
# Create a reference variable for Class TweetTokenizer
tt = TweetTokenizer()
data['tokenized_tweets'] = data['cleaned_tweets'].apply(tt.tokenize)
data['tokenized_tweets'].head()
data

In [ ]:
import nltk
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return text

data['stemmed_tweets']= data['tokenized_tweets'].apply(lambda x: stemming_on_text(x))
#data['stemmed_tweets'].head()


In [ ]:
X = data.cleaned_tweets
y = data.label
y = np.array(y)

from sklearn import metrics
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 2022, stratify = y)

In [ ]:
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X)
X_train_tfidf =  tfidf_vect.transform(X_train)
X_test_tfidf =  tfidf_vect.transform(X_test)

In [ ]:
def train_model(name_of_classifier,classifier, feature_vector_train, label, feature_vector_test):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_test)
    f1_micro = metrics.f1_score(y_test,predictions,average = 'micro')
    f1_macro = metrics.f1_score(y_test,predictions,average = 'macro')
    f1_weighted = metrics.f1_score(y_test,predictions,average = 'weighted')
    accuracy = accuracy_score(y_test,predictions)
    return pd.DataFrame({'model':[name_of_classifier],'f1_micro':[f1_micro],'f1_macro':[f1_macro],'f1_weighted':[f1_weighted],'accuracy':[accuracy]})

In [ ]:
first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),X_train_tfidf, y_train, X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),X_train_tfidf, y_train, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),X_train_tfidf, y_train, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),X_train_tfidf, y_train, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

In [ ]:
ros = RandomOverSampler(random_state=777)
ros_X_train_tfidf, ros_y_train = ros.fit_resample(X_train_tfidf, y_train)

first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ros_X_train_tfidf, ros_y_train, X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),ros_X_train_tfidf, ros_y_train, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),ros_X_train_tfidf, ros_y_train, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),ros_X_train_tfidf, ros_y_train, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

In [ ]:
sm = SMOTE(random_state=777)
sm_X_train_tfidf, sm_y_train = sm.fit_resample(X_train_tfidf, y_train)

first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),sm_X_train_tfidf, sm_y_train, X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),sm_X_train_tfidf, sm_y_train, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),sm_X_train_tfidf, sm_y_train, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),sm_X_train_tfidf, sm_y_train, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

In [ ]:
#ADASYN
ad = ADASYN(random_state=777)
ad_xtrain_tfidf, ad_train_y = ad.fit_resample(X_train_tfidf, y_train)

first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ad_xtrain_tfidf, ad_train_y, X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),ad_xtrain_tfidf, ad_train_y, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),ad_xtrain_tfidf, ad_train_y, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),ad_xtrain_tfidf, ad_train_y, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

In [ ]:
bsm = BorderlineSMOTE()
bsm_xtrain_tfidf, bsm_train_y = bsm.fit_resample(X_train_tfidf, y_train)

first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),bsm_xtrain_tfidf, bsm_train_y, X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),bsm_xtrain_tfidf, bsm_train_y, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),bsm_xtrain_tfidf, bsm_train_y, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),bsm_xtrain_tfidf, bsm_train_y, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

In [ ]:
rus = RandomUnderSampler(random_state=0, replacement=True)
rus_xtrain_tfidf, rus_train_y = rus.fit_resample(X_train_tfidf, y_train)

first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),rus_xtrain_tfidf, rus_train_y ,X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),rus_xtrain_tfidf, rus_train_y, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),rus_xtrain_tfidf, rus_train_y, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),rus_xtrain_tfidf, rus_train_y, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

In [ ]:
tl = TomekLinks()
tl_xtrain_tfidf, tl_train_y = tl.fit_resample(X_train_tfidf, y_train)

first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),tl_xtrain_tfidf, tl_train_y ,X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),tl_xtrain_tfidf, tl_train_y , X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),tl_xtrain_tfidf, tl_train_y, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),tl_xtrain_tfidf, tl_train_y , X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

In [ ]:
!pip install tensorflow-addons==0.16.1
import tensorflow_addons as tfa

In [ ]:
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_addons as tfa

In [ ]:
import nltk
nltk.download('wordnet')

nltk.download('omw-1.4')

In [ ]:
def lemma(data):
    return " ".join([Word(word).lemmatize() for word in data])

data['lemmatized_tweets'] = data['tokenized_tweets'].apply(lambda x: lemma(x))
data


In [ ]:
Y = pd.get_dummies(data.label).values

print('Shape of label tensor:', Y.shape)

In [ ]:
Y

In [ ]:
MAX_NB_WORDS = 500
# Max number of words in each tweets.
MAX_SEQUENCE_LENGTH = 50
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, split=' ') 
tokenizer.fit_on_texts(data.lemmatized_tweets.values)

word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

In [ ]:
X = tokenizer.texts_to_sequences(data.lemmatized_tweets.values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 2022, stratify=Y)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)
f1 = tfa.metrics.F1Score(36,'micro' or 'macro')
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',f1])

epochs = 7
batch_size = 64

In [ ]:
history_1 = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
from tensorflow.keras.optimizers import Adam

seed(1)

tensorflow.random.set_seed(2)
f1 = tfa.metrics.F1Score(36,'micro' or 'macro')
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

opt = Adam(learning_rate=0.0005)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy',f1])

epochs = 7
batch_size = 64

In [ ]:
history_2 = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
plt.figure(figsize=(8, 8))
plt.title('Loss')
plt.plot(history_1.history['loss'], label='train')
plt.plot(history_1.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.figure(figsize=(8, 8))
plt.title('Loss')
plt.plot(history_2.history['loss'], label='train')
plt.plot(history_2.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.figure(figsize=(8, 8))
plt.title('Accuracy')
plt.plot(history_1.history['accuracy'], label='train')
plt.plot(history_1.history['val_accuracy'], label='test')
plt.legend()
plt.show();

# TASK

In [ ]:
!pip install fasttext emoji -q

     |████████████████████████████████| 68 kB 3.8 MB/s 
     |████████████████████████████████| 197 kB 10.8 MB/s 


## Cleaning data

In [ ]:
import fasttext
import sys
import os
import nltk
nltk.download('punkt')

import pandas as pd
import datetime

from bs4 import BeautifulSoup

import re
import itertools
import emoji

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }

def strip_accents(text):
    if 'ø' in text or  'Ø' in text:
        #Do nothing when finding ø 
        return text   
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def tweet_cleaning_for_sentiment_analysis(tweet):    
    
    #Escaping HTML characters
    tweet = BeautifulSoup(tweet).get_text()
    #Special case not handled previously.
    tweet = tweet.replace('\x92',"'")
    #Removal of hastags/account
    #tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)", "USER", tweet).split())
    tweet = tweet.replace("#", "").replace("_", " ")
    #Removal of address
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    #Removal of Punctuation
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    #Lower case
    tweet = tweet.lower()
    #CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS = load_dict_contractions()
    tweet = tweet.replace("’","'")
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    # Standardizing words
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    #Deal with smileys
    #source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    #Deal with emojis
    tweet = emoji.demojize(tweet)
    #Strip accents
    tweet= strip_accents(tweet)
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())
    
    # DO NOT REMOVE STOP WORDS FOR SENTIMENT ANALYSIS - OR AT LEAST NOT NEGATIVE ONES

    return tweet

## Load data

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("train.tsv", delimiter='\t')
data

In [ ]:
data['cleaned_tweets'] = data.tweet.apply(tweet_cleaning_for_sentiment_analysis)
data

## Word embeddings

In [ ]:
import fasttext
import fasttext.util

ModuleNotFoundError: ignored

### Download embedddings

In [ ]:

!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
!unzip crawl-300d-2M-subword.zip
%cd /content

/content/fasttext-embeddings
--2022-07-28 10:32:34--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5828358084 (5.4G) [application/zip]
Saving to: ‘crawl-300d-2M-subword.zip’

crawl-300d-2M-subwo 100%[===================>]   5.43G  37.0MB/s    in 2m 34s  

2022-07-28 10:35:08 (36.2 MB/s) - ‘crawl-300d-2M-subword.zip’ saved [5828358084/5828358084]

Archive:  crawl-300d-2M-subword.zip
  inflating: crawl-300d-2M-subword.vec  
  inflating: crawl-300d-2M-subword.bin  
/content


In [ ]:
!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip crawl-300d-2M.vec.zip

In [ ]:
%cd /content

### Loading model

In [ ]:
fasttext_emb = fasttext.FastText.load_model('./fasttext-embeddings/crawl-300d-2M-subword.bin')

In [ ]:
fasttext_emb.get_nearest_neighbors("🌝")

[(0.9520852565765381, '🌚'),
 (0.9374086856842041, '🌛'),
 (0.9264683723449707, '🌑'),
 (0.9218981266021729, '🌕'),
 (0.921410858631134, '👐'),
 (0.9210396409034729, '🕊'),
 (0.9206425547599792, '🌇'),
 (0.9184413552284241, '🌬'),
 (0.9166590571403503, '🙆'),
 (0.914670467376709, '🕶')]

### Applying word embeddings

In [ ]:
data['sent_vec'] = data.cleaned_tweets.apply(fasttext_emb.get_sentence_vector)

In [ ]:
data

In [ ]:
X = np.array(list(data.sent_vec.apply(lambda x: list(x)).values))

## Training and validation data

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
X = np.array(list(data.sent_vec.apply(lambda x: list(x)).values))

In [ ]:
Xtrain, Xval, ytrain, yval = train_test_split(X, data.label.values, random_state=2022,test_size=0.2, stratify=data.label.values)

In [ ]:
pd.Series(ytrain).value_counts(normalize=True)

In [ ]:
pd.Series(yval).value_counts(normalize=True)

In [ ]:
Xtrain.shape, ytrain.shape

In [ ]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(data['tweet'])

# transform the training and validation data using count vectorizer object

x_count =  count_vect.transform(data['tweet'])

In [ ]:
x_count

In [ ]:
X_count_train, X_count_val, y_count_train, y_count_val = train_test_split(x_count, data.label, random_state=2022,test_size=0.2, stratify=data.label)

In [ ]:
X_count_train.shape

## XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(max_depth=6, n_estimators=1000).fit(X_count_train, y_count_train) 
prediction = xgb_model.predict(Xval) 
f1_score(yval, prediction)

In [ ]:
from sklearn.utils import class_weight
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=ytrain
)

In [ ]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes = np.unique(ytrain),
                                                 y = ytrain)
class_weights

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(max_depth=6, n_estimators=1000).fit(Xtrain, ytrain, sample_weight=classes_weights) 
prediction = xgb_model.predict(Xval)

In [ ]:
prediction = xgb_model.predict(X_count_val) 


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(prediction, y_count_val)) #Count Vectorizer

In [ ]:
print(classification_report(prediction, yval)) #Fasttext

In [ ]:
print(classification_report(prediction, yval)) #Sample weight

In [ ]:
print(classification_report(prediction, yval)) #Sample weight

In [ ]:
import xgboost as xgb

In [ ]:
train = xgb.DMatrix(Xtrain, ytrain)
test = xgb.DMatrix(Xval, yval)

# We need to define parameters as dict
params = {
    "learning_rate": 0.01,
    "max_depth": 3
}
# training, we set the early stopping rounds parameter
model_xgb = xgb.train(params, 
          train, evals=[(train, "train"), (test, "validation")], 
          num_boost_round=100, early_stopping_rounds=20)

In [ ]:
model_xgb.best_ntree_limit

In [ ]:
predictions = model_xgb.predict(test, ntree_limit=model_xgb.best_ntree_limit)

In [ ]:
print(classification_report(prediction, yval)) #Sample weight

In [ ]:
xgb_model.save_model("xgboost_model.json")

In [ ]:
from pprint import pprint

In [ ]:
def randomsearch():
  n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 15)]
  # Number of features to consider at every split
  max_features = ['auto', 'sqrt']
  # Maximum number of levels in tree
  max_depth = [int(x) for x in np.linspace(10, 200, num = 10)]
  max_depth.append(None)
  # Minimum number of samples required to split a node
  min_samples_split = [2, 4, 8, 16,32]
  # Minimum number of samples required at each leaf node
  min_samples_leaf = [1, 2, 4,8,16]
  # Method of selecting samples for training each tree
  bootstrap = [True, False]# Create the random grid
  random_grid = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'bootstrap': bootstrap}
                
  pprint(random_grid,compact=True)
  
  return random_grid

_ =randomsearch()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_grid = randomsearch()


rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=4, random_state=2022, n_jobs = -1)# Fit the random search model
rf_random.fit(Xtrain, ytrain)


best_random = rf_random.best_estimator_

print("Paramters:")
pprint(rf_random.best_params_)

print("Training")
print(classification_report(ytrain,best_random.predict(Xtrain)))

print()

print("Testing")
print(classification_report(yval,best_random.predict(Xval)))

In [ ]:
 rf = RandomForestClassifier(max_features='log2', class_weight='balanced', random_state=2022, n_jobs=-1)
 rf.fit(Xtrain, ytrain)

In [ ]:
predictions = rf.predict(Xval)

In [ ]:
print(classification_report(predictions, yval)) #Random forest

In [ ]:
params = {
    'n_estimators':[500],
    'min_child_weight':[4,5], 
    'gamma':[i/10.0 for i in range(3,6)],  
    'subsample':[i/10.0 for i in range(6,11)],
    'colsample_bytree':[i/10.0 for i in range(6,11)], 
    'max_depth': [2,3,4,6,7],
    'objective': ['reg:squarederror', 'reg:tweedie'],
    'booster': ['gbtree', 'gblinear'],
    'eval_metric': ['rmse'],
    'eta': [i/10.0 for i in range(3,6)],
}

reg = XGBRegressor(nthread=-1)

# run randomized search
n_iter_search = 100
random_search = RandomizedSearchCV(reg, param_distributions=params,
                                   n_iter=n_iter_search, cv=3, iid=False, scoring='neg_mean_squared_error')



## Test data

In [ ]:
test = pd.read_csv("test.tsv", delimiter='\t')
test

In [ ]:
test.label.value_counts()

In [ ]:
test['cleaned_tweets'] = test.tweet.apply(tweet_cleaning_for_sentiment_analysis)
#test['sent_vec'] = test.cleaned_tweets.apply(fasttext_emb.get_sentence_vector)

In [ ]:
X_test = np.array(list(test.sent_vec.apply(lambda x: list(x)).values))
y_test = test.label.values

In [ ]:
X_test.shape

### Random Forest

In [ ]:
predictions = rf.predict(X_test)
print(classification_report(predictions, y_test)) #Random forest

### XGBoost

In [ ]:
predictions = xgb_model.predict(X_test)
print(classification_report(predictions, y_test)) #xgboost

### BERTweet

In [ ]:
test['BERTweet'] = test['cleaned_tweets'].apply(sa1.inference)

In [ ]:
test

In [ ]:
test['BERTweet'] = test['BERTweet'].replace('NEUTRAL', 0)
test['BERTweet'] = test['BERTweet'].replace('NEGATIVE', -1)
test['BERTweet'] = test['BERTweet'].replace('POSITIVE', 1)

test

In [ ]:
print(classification_report(test.label, test.BERTweet)) #BERTweet

### BERT base

In [ ]:
test['bert-base-multilingual-uncased-sentiment_clean'] = test['cleaned_tweets'].apply(sa2.inference)

In [ ]:
test

In [ ]:
test['bert-base-multilingual-uncased-sentiment_clean'] = test['bert-base-multilingual-uncased-sentiment_clean'].replace('NEUTRAL', 0)
test['bert-base-multilingual-uncased-sentiment_clean'] = test['bert-base-multilingual-uncased-sentiment_clean'].replace('NEGATIVE', -1)
test['bert-base-multilingual-uncased-sentiment_clean'] = test['bert-base-multilingual-uncased-sentiment_clean'].replace('POSITIVE', 1)

test

In [ ]:
print(classification_report(test.label, test['bert-base-multilingual-uncased-sentiment_clean'])) #BERTweet

## Benchmark dataset

In [ ]:
benchmark_df = pd.read_excel("Sentiment models.xlsx", engine='openpyxl')
benchmark_df

In [ ]:
benchmark_df = benchmark_df[['Tweets']]

In [ ]:
benchmark_df['cleaned_tweets'] = benchmark_df.Tweets.apply(tweet_cleaning_for_sentiment_analysis)
benchmark_df['sent_vec'] = benchmark_df.cleaned_tweets.apply(fasttext_emb.get_sentence_vector)

In [ ]:
benchmark_df

In [ ]:
X = np.array(list(benchmark_df.sent_vec.apply(lambda x: list(x)).values))
benchmark_df['Random Forest'] = rf.predict(X)

In [ ]:
benchmark_df['Random Forest'] = benchmark_df['Random Forest'].replace(0, 'Neutral')
benchmark_df['Random Forest'] = benchmark_df['Random Forest'].replace(-1, 'Negative')
benchmark_df['Random Forest'] = benchmark_df['Random Forest'].replace(1, 'Positive')

benchmark_df

In [ ]:
benchmark_df['XGBoost'] = xgb_model.predict(X) 
benchmark_df

In [ ]:
benchmark_df['XGBoost'] = benchmark_df['XGBoost'].replace(0, 'Neutral')
benchmark_df['XGBoost'] = benchmark_df['XGBoost'].replace(-1, 'Negative')
benchmark_df['XGBoost'] = benchmark_df['XGBoost'].replace(1, 'Positive')

benchmark_df

In [ ]:
benchmark_df.to_excel("Sentiment.xlsx", index=False)

# Dataset curation

In [ ]:
import pandas as pd
import requests

In [ ]:
train = []
test = []
dev = []

## SST-5

In [ ]:
%%bash 
mkdir SST-5
cd SST-5
wget https://raw.githubusercontent.com/prrao87/fine-grained-sentiment/master/data/sst/sst_train.txt
wget https://raw.githubusercontent.com/prrao87/fine-grained-sentiment/master/data/sst/sst_dev.txt
wget https://raw.githubusercontent.com/prrao87/fine-grained-sentiment/master/data/sst/sst_test.txt
cd /content

In [ ]:
!pip install pytreebank

In [ ]:
f = open("/content/SST-5/sst_train.txt").readlines()
f

In [ ]:
def relabel(label):
  if int(label[-1]) > 3:
    return "POSITIVE"
  elif int(label[-1]) == 3:
    return "NEUTRAL"
  else:
    return "NEGATIVE"

In [ ]:
sst5train = pd.read_csv("/content/SST-5/sst_train.txt", delimiter='\t', names=['label', 'tweet'])
sst5test = pd.read_csv("/content/SST-5/sst_test.txt", delimiter='\t', names=['label', 'tweet'])
sst5dev = pd.read_csv("/content/SST-5/sst_dev.txt", delimiter='\t', names=['label', 'tweet'])

In [ ]:
sst5train['label'] = sst5train['label'].apply(relabel)
sst5test['label'] = sst5test['label'].apply(relabel)
sst5dev['label'] = sst5dev['label'].apply(relabel)

In [ ]:
print(sst5train['label'].value_counts())
print(sst5test['label'].value_counts())
print(sst5dev['label'].value_counts())

In [ ]:
train.append(sst5train)
test.append(sst5test)
dev.append(sst5dev)

## Multi domain sentiment analysis

In [ ]:
!wget https://www.cs.jhu.edu/~mdredze/datasets/sentiment/processed_stars.tar.gz
!tar -xf processed_stars.tar.gz

In [ ]:
!wget https://www.cs.jhu.edu/~mdredze/datasets/sentiment/unprocessed.tar.gz
!tar -xf unprocessed.tar.gz

In [ ]:
!tar -xf unprocessed.tar.gz.1

In [ ]:
import pickle

In [ ]:
import os
path_to_data = "./sorted_data"
# path_to_processed_data = "processed"
dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        dirs.append(entry.path)
dirs

In [ ]:
!pip install --upgrade xml-python

In [ ]:
import xml.etree.ElementTree as ET
import re 

# with open("./sorted_data/apparel/unlabeled.review", encoding='iso-8859-5') as f:
#     xml = f.read()
# tree = ET.fromstring(re.sub(r"(<\?xml[^>]+\?>)", r"\1<review>", xml) + "</review>")

tree = ET.parse('/content/processed_stars/books/all_balanced.review', parser = ET.XMLParser(encoding = 'iso-8859-5'))

In [ ]:
with open("./sorted_data/apparel/unlabeled.review", encoding='iso-8859-5') as f:
  xml = f.read()


data = [(i + "</review>").strip("\n") for i in xml.split("</review>")]

In [ ]:
from io import StringIO
from lxml import etree
parser = lxml.etree.XMLParser(encoding='utf-8', recover=True)

def generate_trees(filepath):

  with open(filepath, encoding='iso-8859-5') as f:
    xml = f.read()


  data = [(i + "</review>").strip("\n") for i in xml.split("</review>")]
  data = data[:-1]
  
  data_trees = []
  
  for i, d in enumerate(data):
    #print(i)
    with StringIO(d) as f:
      #tree = ET.parse(f, parser = ET.XMLParser(encoding = 'utf-8-sig'))
      tree = ET.parse(f, parser)
    
    data_trees.append(tree)

  return data_trees

In [ ]:
def get_row(tree):
  root = tree.getroot()
  cols = ["unique_id", "asin", "product_name", "product_type", "helpful", "rating", "title", "date", "reviewer", "reviewer_location", "review_text"]
  rows = []

  try:
    unique_id = root.find("unique_id").text
  except:
    unique_id = ''

  try:  
    asin = root.find("asin").text
  except:
    asin = ''

  try:
    product_name = root.find("product_name").text
  except:
    product_name = ''

  try:
    product_type = root.find("product_type").text
  except:
    product_type = ''
  
  try:
    helpful = root.find("helpful").text
  except:
    helpful = ''

  try:
    rating = root.find("rating").text
  except:
    rating = ''

  try:  
    title = root.find("title").text
  except:
    title = ''

  try:  
    date = root.find("date").text
  except:
    date = ''

  try:  
    reviewer = root.find("reviewer").text
  except:
    reviewer = ''
  
  try:
    reviewer_location = root.find("reviewer_location").text
  except:
    reviewer_location = ''

  try:  
    review_text = root.find("review_text").text 
  except:
    review_text= ''

  rows.append({"unique_id": unique_id,
                "asin": asin,
                "product_name": product_name,
                "product_type": product_type,
                "helpful": helpful, 
                "rating": rating, 
                "title": title, 
                "date": date, 
                "reviewer": reviewer, 
                "reviewer_location": reviewer_location, 
                "review_text": review_text})

  df = pd.DataFrame(rows, columns=cols)
  
  return df

In [ ]:
def get_row(tree):
  root = tree.getroot()
  cols = [ "asin", "product_type", "helpful", "rating", "title",  "review_text"]
  rows = []

 
  try:  
    asin = root.find("asin").text
  except:
    asin = ''

  try:
    product_type = root.find("product_type").text
  except:
    product_type = ''
  
  try:
    helpful = root.find("helpful").text
  except:
    helpful = ''

  try:
    rating = root.find("rating").text
  except:
    rating = ''

  try:  
    title = root.find("title").text
  except:
    title = ''

  try:  
    review_text = root.find("review_text").text 
  except:
    review_text= ''

  rows.append({
                "asin": asin,  
                "product_type": product_type,
                "helpful": helpful, 
                "rating": rating, 
                "title": title, 
                "review_text": review_text})

  #df = pd.DataFrame(rows, columns=cols)
  
  return rows

In [ ]:
def strip_line_breaks(df):
    for i in df.columns:
        df[i] = df[i].apply(lambda x : x.strip('\n'))
    return df

In [ ]:
def generate_df_from_trees(data_trees):
  data = pd.DataFrame()
  data_rows = []
  for tree in tqdm(data_trees):
    row = get_row(tree)
    data_rows.extend(row)
    
  data = pd.DataFrame(data_rows)

  return strip_line_breaks(data)

In [ ]:
def parse_file(filepath):
  data_trees = generate_trees(filepath)
  data = generate_df_from_trees(data_trees)

  return data

In [ ]:
from tqdm import tqdm

In [ ]:
filepath = "./sorted_data/video/all.review"
test = parse_file(filepath)


In [ ]:
path_to_processed_data = "./multi-domain/processed"

In [ ]:
import os
path_to_data = "./sorted_data"
# path_to_processed_data = "processed"
dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        dirs.append(entry.path)
dirs

In [ ]:
import shutil
if os.path.exists(path_to_processed_data):
     shutil.rmtree(path_to_processed_data)
os.mkdir(path_to_processed_data)

In [ ]:
processed_dirs = []
processed_file_names_dict = {}
for dir in dirs[2:]:
    product_type = dir.split('/')[-1]
    processed_file_names_dict[product_type] = []
    for entry in os.scandir(dir):
        print(f"Running {entry.path}") 
        try:
          df = parse_file(entry.path)
          
          if tree is not None:
              processed_folder = path_to_processed_data + '/' + product_type
              if not os.path.exists(processed_folder):
                  os.makedirs(processed_folder)
                  processed_dirs.append(processed_folder)
                  print("Generating processed files in folder: ",processed_folder)
              
              processed_file_name = product_type + "_" + ((entry.path).split('/')[-1]).split('.review')[0] + ".csv"
              processed_file_path = processed_folder + "/" + processed_file_name
              df.to_csv(processed_file_path, index=False)
              processed_file_names_dict[product_type].append(processed_file_path)
        except Exception as e:
          print(e)
                              

In [ ]:
!zip -r multi-domain.zip /content/multi-domain

In [ ]:
import pandas as pd

d = pd.read_csv("/content/multi-domain/processed/baby/baby_all.csv")

In [ ]:
d.rating.value_counts()

In [ ]:
import lxml.etree
import xml.etree.ElementTree as ET

parser = lxml.etree.XMLParser(encoding='utf-8', recover=True)

## Sentiment Analysis on Financial news

In [ ]:
!mkdir financial-news

In [ ]:
financial_news = pd.read_csv("./financial-news/all-data.csv", names=['label', 'tweet'], encoding='iso-8859-5')
financial_news

In [ ]:
financial_news['label'] = financial_news['label'].apply(str.upper)

In [ ]:
financial_news['label'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
train_ratio = 0.7
validation_ratio = 0.1
test_ratio = 0.2

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
financial_news_train, financial_news_test = train_test_split(financial_news, test_size=1 - train_ratio, stratify=financial_news.label.values)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
financial_news_dev, financial_news_test = train_test_split(financial_news_test, test_size=test_ratio/(test_ratio + validation_ratio), stratify=financial_news_test.label.values) 

print(financial_news_train, financial_news_dev, financial_news_test)

In [ ]:
financial_news_test

In [ ]:
print(financial_news_train['label'].value_counts())
print(financial_news_test['label'].value_counts())
print(financial_news_dev['label'].value_counts())

In [ ]:
train.append(financial_news_train)
test.append(financial_test)
dev.append(financial_dev)

## Financial Sentiment Analysis

In [ ]:
!mkdir financial-sentiment

In [ ]:
financial = pd.read_csv("./financial-sentiment/data.csv", names=['tweet', 'label'], skiprows=1)
financial

In [ ]:
financial['label'] = financial['label'].apply(str.upper)

In [ ]:
financial['label'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
train_ratio = 0.7
validation_ratio = 0.1
test_ratio = 0.2

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
financial_train, financial_test = train_test_split(financial, test_size=1 - train_ratio, stratify=financial.label.values)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
financial_dev, financial_test = train_test_split(financial_test, test_size=test_ratio/(test_ratio + validation_ratio), stratify=financial_test.label.values) 

print(financial_train, financial_dev, financial_test)

In [ ]:
financial_test

In [ ]:
print(financial_train['label'].value_counts())
print(financial_test['label'].value_counts())
print(financial_dev['label'].value_counts())

In [ ]:
train.append(financial_train)
test.append(financial_test)
dev.append(financial_dev)

## CovidSenti

In [ ]:
!mkdir CovidSenti
%cd CovidSenti
!wget https://raw.githubusercontent.com/usmaann/COVIDSenti/main/COVIDSenti.csv

In [ ]:
%cd /content

In [ ]:
covidsenti = pd.read_csv("./CovidSenti/COVIDSenti.csv")
covidsenti

In [ ]:
def relabel(label):
  if label=="neu":
    return "NEUTRAL"
  elif label=="pos":
    return "POSITIVE"
  else:
    return "NEGATIVE"

In [ ]:
covidsenti['label'] = covidsenti['label'].apply(relabel)

In [ ]:
covidsenti['label'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
train_ratio = 0.7
validation_ratio = 0.1
test_ratio = 0.2

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
covidsenti_train, covidsenti_test = train_test_split(covidsenti, test_size=1 - train_ratio, stratify=covidsenti.label.values)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
covidsenti_dev, covidsenti_test = train_test_split(covidsenti_test, test_size=test_ratio/(test_ratio + validation_ratio), stratify=covidsenti_test.label.values) 

print(covidsenti_train, covidsenti_dev, covidsenti_test)

In [ ]:
print(covidsenti_train['label'].value_counts())
print(covidsenti_test['label'].value_counts())
print(covidsenti_dev['label'].value_counts())

In [ ]:
train.append(covidsenti_train)
test.append(covidsenti_test)
dev.append(covidsenti_dev)

In [ ]:
len(train), len(test), len(dev)

In [ ]:
!mkdir data-save

In [ ]:
import pickle

with open("./data-save/train.pkl","wb") as f:
  pickle.dump(train, f)

with open("./data-save/test.pkl","wb") as f:
  pickle.dump(test, f)

with open("./data-save/dev.pkl","wb") as f:
  pickle.dump(dev, f)

In [ ]:
!zip -r data-save.zip ./data-save

## IMDB

In [ ]:
test_new_df.to_csv("data-save-added/test-added.csv", index=False, encoding='UTF-8')
dev_new_df.to_csv("data-save-added/dev-added.csv", index=False, encoding='UTF-8')

# TASK : Collated data sentiment analysis training

In [ ]:
!pip install fasttext emoji -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/Internship/Coffeebeans/data/finaldata.zip finaldata.zip
!unzip finaldata.zip

Archive:  finaldata.zip
  inflating: dev.csv                 
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
!unzip /content/drive/MyDrive/Internship/Coffeebeans/data/pickled_data.zip -d /content

Archive:  /content/drive/MyDrive/Internship/Coffeebeans/data/pickled_data.zip
replace /content/content/pickled_data/dev.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!unzip /content/drive/MyDrive/Internship/Coffeebeans/data/finaldata.zip -d /content

Archive:  /content/drive/MyDrive/Internship/Coffeebeans/data/finaldata.zip
  inflating: /content/dev.csv        
  inflating: /content/test.csv       
  inflating: /content/train.csv      


## Cleaning data

In [ ]:
import fasttext
import sys
import os
import nltk
nltk.download('punkt')

import pandas as pd
import datetime

from bs4 import BeautifulSoup

import re
import itertools
import emoji

import numpy as np
import pickle

import fasttext
import fasttext.util

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }

def strip_accents(text):
    if 'ø' in text or  'Ø' in text:
        #Do nothing when finding ø 
        return text   
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def tweet_cleaning_for_sentiment_analysis(tweet):    
    
    #Escaping HTML characters
    tweet = BeautifulSoup(tweet).get_text()
    #Special case not handled previously.
    tweet = tweet.replace('\x92',"'")
    #Removal of hastags/account
    #tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)", "USER", tweet).split())
    tweet = tweet.replace("#", "").replace("_", " ")
    #Removal of address
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    #Removal of Punctuation
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    #Lower case
    tweet = tweet.lower()
    #CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS = load_dict_contractions()
    tweet = tweet.replace("’","'")
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    # Standardizing words
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    #Deal with smileys
    #source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    #Strip accents
    tweet= strip_accents(tweet)
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())
    
    # DO NOT REMOVE STOP WORDS FOR SENTIMENT ANALYSIS - OR AT LEAST NOT NEGATIVE ONES

    return tweet

## Word embeddings

### Download embedddings

In [ ]:
!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
!unzip crawl-300d-2M-subword.zip
%cd /content

/content/fasttext-embeddings
--2022-07-21 14:55:23--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5828358084 (5.4G) [application/zip]
Saving to: ‘crawl-300d-2M-subword.zip’

crawl-300d-2M-subwo 100%[===================>]   5.43G  32.1MB/s    in 3m 13s  

2022-07-21 14:58:37 (28.8 MB/s) - ‘crawl-300d-2M-subword.zip’ saved [5828358084/5828358084]

Archive:  crawl-300d-2M-subword.zip
  inflating: crawl-300d-2M-subword.vec  
  inflating: crawl-300d-2M-subword.bin  
/content


In [ ]:
!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.vec.zip
!unzip wiki-news-300d-1M-subword.vec.zip
%cd /content

mkdir: cannot create directory ‘fasttext-embeddings’: File exists
/content/fasttext-embeddings
--2022-07-21 15:24:21--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 587879973 (561M) [application/zip]
Saving to: ‘wiki-news-300d-1M-subword.vec.zip’

wiki-news-300d-1M-s 100%[===================>] 560.65M  36.8MB/s    in 16s     

2022-07-21 15:24:38 (34.7 MB/s) - ‘wiki-news-300d-1M-subword.vec.zip’ saved [587879973/587879973]

Archive:  wiki-news-300d-1M-subword.vec.zip
  inflating: wiki-news-300d-1M-subword.vec  
/content


In [ ]:
!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gzip -d cc.en.300.bin.gz
%cd /content

mkdir: cannot create directory ‘fasttext-embeddings’: File exists
/content/fasttext-embeddings
--2022-07-21 15:28:31--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  28.1MB/s    in 2m 18s  

2022-07-21 15:30:49 (31.2 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]

/content


In [ ]:
# !mkdir fasttext-embeddings
# %cd fasttext-embeddings
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
# !unzip crawl-300d-2M.vec.zip

In [ ]:
%cd /content

/content


### Loading model

In [ ]:

# fasttext_emb = fasttext.load_model('/content/fasttext-embeddings/cc.en.300.bin')
# print(ft.get_dimension())

# fasttext.util.reduce_model(ft, 100)
# print(ft.get_dimension())

NameError: ignored

In [ ]:
fasttext_emb = fasttext.FastText.load_model('./fasttext-embeddings/crawl-300d-2M-subword.bin')
#fasttext_emb = fasttext.FastText.load_model('./fasttext-embeddings/wiki-news-300d-1M-subword.vec')

In [ ]:
fasttext_emb.get_nearest_neighbors("__user__")

[(0.6122851371765137, 'intrebari'),
 (0.6034965515136719, 'vmware-modconfig'),
 (0.5972209572792053, 'eclipselink-orm.xml'),
 (0.5965851545333862, 'base-config'),
 (0.5962321162223816, 'hostnamectl'),
 (0.5956684350967407, 'myconfig'),
 (0.593761146068573, 'webpack.config.js'),
 (0.5922401547431946, 'mkvirtualenv'),
 (0.5871965289115906, 'libxml2-dev'),
 (0.5849608182907104, 'jboss-service.xml')]

## Prepare data

In [ ]:
# def prepare_data(filepath, embedding):
#   data = pd.read_csv(filepath)
#   data['cleaned_tweets'] = data.tweet.astype(str).apply(tweet_cleaning_for_sentiment_analysis)
#   data['sent_vec'] = data.cleaned_tweets.apply(embedding.get_sentence_vector)

#   data = data[['cleaned_tweets', 'sent_vec', 'label']]

#   X = data.sent_vec.apply(lambda x: list(x)).values
  
#   return X, data.label.values

In [ ]:
data = pd.read_csv("train.csv")
data['cleaned_tweets'] = data.tweet.astype(str).apply(tweet_cleaning_for_sentiment_analysis)
data['sent_vec'] = data.cleaned_tweets.apply(fasttext_emb.get_sentence_vector)


with open("./pickled_data/train.pkl", "wb") as f:
  pickle.dump(data, f)

del data

In [ ]:
data = pd.read_csv("dev.csv")
data['cleaned_tweets'] = data.tweet.astype(str).apply(tweet_cleaning_for_sentiment_analysis)
data['sent_vec'] = data.cleaned_tweets.apply(fasttext_emb.get_sentence_vector)

with open("./pickled_data/dev.pkl", "wb") as f:
  pickle.dump(data, f)

del data

In [ ]:
data = pd.read_csv("test.csv")
data['cleaned_tweets'] = data.tweet.astype(str).apply(tweet_cleaning_for_sentiment_analysis)
data['sent_vec'] = data.cleaned_tweets.apply(fasttext_emb.get_sentence_vector)

with open("./pickled_data/test.pkl", "wb") as f:
  pickle.dump(data, f)

del data

In [ ]:
def prepare_data(pickle_filepath):
  data = pickle.load(open(pickle_filepath,'rb'))
  
  X = np.array(list(data.sent_vec.apply(lambda x: list(x)).values))
  
  return X, data.label.values

In [ ]:
!zip -r pickled_data.zip /content/pickled_data
!cp pickled_data.zip /content/drive/MyDrive/Internship/Coffeebeans/data/

	zip warning: name not matched: /content/pickled_data

zip error: Nothing to do! (try: zip -r pickled_data.zip . -i /content/pickled_data)
cp: cannot stat 'pickled_data.zip': No such file or directory


In [ ]:
!gdown https://drive.google.com/file/d/1KyBIlhLCH4RE6iEWTXRhoiAMe4KwUPFU/view?usp=sharing --fuzzy
!unzip pickled_data.zip

Downloading...
From: https://drive.google.com/uc?id=1KyBIlhLCH4RE6iEWTXRhoiAMe4KwUPFU
To: /content/pickled_data.zip
100% 226M/226M [00:01<00:00, 191MB/s]
Archive:  pickled_data.zip
   creating: content/pickled_data/
  inflating: content/pickled_data/dev.pkl  
  inflating: content/pickled_data/test.pkl  
  inflating: content/pickled_data/train.pkl  


In [ ]:
Xtrain, ytrain = prepare_data("./content/pickled_data/train.pkl")
Xdev, ydev = prepare_data("./content/pickled_data/dev.pkl")

In [ ]:
pd.Series(ytrain).value_counts(
  
).to_dict()

{'NEGATIVE': 29782, 'NEUTRAL': 52999, 'POSITIVE': 24242}

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
sampling = {'NEGATIVE': 29782, 'NEUTRAL': 29000, 'POSITIVE': 24242}

under = RandomUnderSampler(sampling_strategy=sampling, random_state=2022)# first performing oversampling to minority class



Xtrain_under, ytrain_under = under.fit_resample(Xtrain, ytrain)

## Training and validation data

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
# Xtrain, Xval, ytrain, yval = train_test_split(X, data.label.values, random_state=2022,test_size=0.2, stratify=data.label.values)

In [ ]:
pd.Series(ytrain).value_counts(normalize=True)

NEUTRAL     0.495211
NEGATIVE    0.278277
POSITIVE    0.226512
dtype: float64

In [ ]:
pd.Series(ydev).value_counts(normalize=True)

NEUTRAL     0.497282
NEGATIVE    0.277425
POSITIVE    0.225293
dtype: float64

In [ ]:
Xtrain.shape, ytrain.shape

((107023, 300), (107023,))

In [ ]:
# count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
# count_vect.fit(data['tweet'])

# # transform the training and validation data using count vectorizer object

# x_count =  count_vect.transform(data['tweet'])

# X_count_train, X_count_val, y_count_train, y_count_val = train_test_split(x_count, data.label, random_state=2022,test_size=0.2, stratify=data.label)

# X_count_train.shape

## XGBoost

In [ ]:
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

from xgboost import XGBClassifier

In [ ]:
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=ytrain
)

In [ ]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes = np.unique(ytrain),
                                                 y = ytrain)
class_weights

array([1.19784881, 0.67311333, 1.471592  ])

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(max_depth=7, 
                          n_estimators=1000, 
                          gamma=0.5,
                          min_child_weight=5,
                          eta=0.5,
                          random_state=2022,
                          tree_method = "gpu_hist", 
                          single_precision_histogram=True)

# 'n_estimators':[500],
#     'min_child_weight':[4,5], 
#     'gamma':[i/10.0 for i in range(3,6)],  
#     'subsample':[i/10.0 for i in range(6,11)],
#     'colsample_bytree':[i/10.0 for i in range(6,11)], 
#     'max_depth': [2,3,4,6,7],
#     'objective': ['reg:squarederror', 'reg:tweedie'],
#     'booster': ['gbtree', 'gblinear'],
#     'eval_metric': ['rmse'],
#     'eta': [i/10.0 for i in range(3,6)],

xgb_model.fit(Xtrain, ytrain, sample_weight=classes_weights) 


XGBClassifier(eta=0.5, gamma=0.5, max_depth=7, min_child_weight=5,
              n_estimators=1000, objective='multi:softprob', random_state=2022,
              single_precision_histogram=True, tree_method='gpu_hist')

In [ ]:
prediction = xgb_model.predict(Xdev)
print(classification_report(prediction, ydev)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.68      0.78      0.73      3701
     NEUTRAL       0.92      0.82      0.87      8473
    POSITIVE       0.74      0.82      0.78      3095

    accuracy                           0.81     15269
   macro avg       0.78      0.81      0.79     15269
weighted avg       0.82      0.81      0.82     15269



In [ ]:
prediction = xgb_model.predict(Xdev)
print(classification_report(prediction, ydev)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.67      0.79      0.73      3593
     NEUTRAL       0.92      0.82      0.87      8609
    POSITIVE       0.73      0.82      0.78      3067

    accuracy                           0.81     15269
   macro avg       0.78      0.81      0.79     15269
weighted avg       0.83      0.81      0.82     15269



In [ ]:
# XGBClassifier(max_depth=6, n_estimators=1000, objective='multi:softprob',
#               single_precision_histogram=True, tree_method='gpu_hist')

prediction = xgb_model.predict(Xdev)
print(classification_report(ydev, prediction)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.78      0.68      0.72      4236
     NEUTRAL       0.82      0.91      0.86      7593
    POSITIVE       0.81      0.74      0.77      3440

    accuracy                           0.81     15269
   macro avg       0.80      0.78      0.79     15269
weighted avg       0.81      0.81      0.80     15269



In [ ]:
# XGBClassifier(eta=0.5, gamma=0.5, max_depth=7, min_child_weight=5,
#               n_estimators=1000, objective='multi:softprob',
#               single_precision_histogram=True, tree_method='gpu_hist')

prediction = xgb_model.predict(Xdev)
print(classification_report(ydev, prediction)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.80      0.67      0.73      4236
     NEUTRAL       0.82      0.93      0.87      7593
    POSITIVE       0.82      0.73      0.78      3440

    accuracy                           0.81     15269
   macro avg       0.81      0.78      0.79     15269
weighted avg       0.81      0.81      0.81     15269



In [ ]:
prediction = xgb_model.predict(Xdev)
print(classification_report(ydev, prediction)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.79      0.68      0.73      4236
     NEUTRAL       0.82      0.92      0.87      7593
    POSITIVE       0.82      0.73      0.78      3440

    accuracy                           0.81     15269
   macro avg       0.81      0.78      0.79     15269
weighted avg       0.81      0.81      0.81     15269



In [ ]:
prediction = xgb_model.predict(Xdev)
print(classification_report(ydev, prediction)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.77      0.68      0.72      4236
     NEUTRAL       0.83      0.90      0.86      7593
    POSITIVE       0.79      0.74      0.77      3440

    accuracy                           0.80     15269
   macro avg       0.79      0.77      0.78     15269
weighted avg       0.80      0.80      0.80     15269



In [ ]:
xgb_model.save_model("./models/finaldata_xgboost_model.json")

In [ ]:
print(classification_report(prediction, y_count_val)) #Count Vectorizer

In [ ]:
print(classification_report(prediction, yval)) #Fasttext

In [ ]:
print(classification_report(prediction, yval)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.66      0.78      0.72      5028
     NEUTRAL       0.92      0.81      0.86     12046
    POSITIVE       0.72      0.81      0.76      4331

    accuracy                           0.80     21405
   macro avg       0.77      0.80      0.78     21405
weighted avg       0.82      0.80      0.81     21405



In [ ]:
print(classification_report(prediction, yval)) #Sample weight

In [ ]:
from pprint import pprint

In [ ]:
def randomsearch():
  n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 15)]
  # Number of features to consider at every split
  max_features = ['auto', 'sqrt']
  # Maximum number of levels in tree
  max_depth = [int(x) for x in np.linspace(10, 200, num = 10)]
  max_depth.append(None)
  # Minimum number of samples required to split a node
  min_samples_split = [2, 4, 8, 16,32]
  # Minimum number of samples required at each leaf node
  min_samples_leaf = [1, 2, 4,8,16]
  # Method of selecting samples for training each tree
  bootstrap = [True, False]# Create the random grid
  random_grid = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'bootstrap': bootstrap}
                
  pprint(random_grid,compact=True)
  
  return random_grid

_ =randomsearch()

In [ ]:
import pickle
pickle.dump(xgb_model, open("/content/models/finaldata_xgboost_model.pkl", "wb"))

import joblib
joblib.dump(xgb_model, "/content/models/finaldata_xgboost_model.joblib") 

['/content/models/finaldata_xgboost_model.joblib']

## XGBoost - Undersampling

In [ ]:
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

from xgboost import XGBClassifier

In [ ]:
classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=ytrain_under
)

In [ ]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes = np.unique(ytrain),
                                                 y = ytrain_under)
class_weights

array([0.92924138, 0.95429885, 1.14159998])

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(max_depth=8, 
                          n_estimators=1000, 
                          gamma=0.5,
                          min_child_weight=5,
                          eta=0.5,
                          random_state=2022,
                          tree_method = "gpu_hist", 
                          single_precision_histogram=True)

# 'n_estimators':[500],
#     'min_child_weight':[4,5], 
#     'gamma':[i/10.0 for i in range(3,6)],  
#     'subsample':[i/10.0 for i in range(6,11)],
#     'colsample_bytree':[i/10.0 for i in range(6,11)], 
#     'max_depth': [2,3,4,6,7],
#     'objective': ['reg:squarederror', 'reg:tweedie'],
#     'booster': ['gbtree', 'gblinear'],
#     'eval_metric': ['rmse'],
#     'eta': [i/10.0 for i in range(3,6)],

xgb_model.fit(Xtrain_under, ytrain_under) 


XGBClassifier(eta=0.5, gamma=0.5, max_depth=8, min_child_weight=5,
              n_estimators=1000, objective='multi:softprob', random_state=2022,
              single_precision_histogram=True, tree_method='gpu_hist')

In [ ]:
xgb_model

XGBClassifier(eta=0.5, gamma=0.5, max_depth=8, min_child_weight=5,
              n_estimators=1000, objective='multi:softprob', random_state=2022,
              single_precision_histogram=True, tree_method='gpu_hist')

In [ ]:
# XGBClassifier(eta=0.5, gamma=0.5, max_depth=7, min_child_weight=5,
#               n_estimators=1000, objective='multi:softprob', random_state=2022,
#               single_precision_histogram=True, tree_method='gpu_hist')
prediction = xgb_model.predict(Xdev)
print(classification_report(prediction, ydev)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.73      0.74      0.74      4182
     NEUTRAL       0.88      0.84      0.86      7929
    POSITIVE       0.74      0.81      0.77      3158

    accuracy                           0.81     15269
   macro avg       0.78      0.80      0.79     15269
weighted avg       0.81      0.81      0.81     15269



In [ ]:
# XGBClassifier(eta=0.5, gamma=0.5, max_depth=5, min_child_weight=7,
#               n_estimators=1000, objective='multi:softprob', random_state=2022,
#               single_precision_histogram=True, tree_method='gpu_hist')

prediction = xgb_model.predict(Xdev)
print(classification_report(prediction, ydev)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.72      0.73      0.73      4195
     NEUTRAL       0.87      0.84      0.85      7806
    POSITIVE       0.75      0.79      0.77      3268

    accuracy                           0.80     15269
   macro avg       0.78      0.79      0.78     15269
weighted avg       0.80      0.80      0.80     15269



In [ ]:
# XGBClassifier(eta=0.5, gamma=0.5, max_depth=8, min_child_weight=5,
#               n_estimators=1000, objective='multi:softprob', random_state=2022,
#               single_precision_histogram=True, tree_method='gpu_hist')

prediction = xgb_model.predict(Xdev)
print(classification_report(ydev, prediction)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.75      0.73      0.74      4236
     NEUTRAL       0.84      0.89      0.86      7593
    POSITIVE       0.81      0.74      0.78      3440

    accuracy                           0.81     15269
   macro avg       0.80      0.79      0.79     15269
weighted avg       0.81      0.81      0.81     15269



In [ ]:
# XGBClassifier(eta=0.5, gamma=0.5, max_depth=7, min_child_weight=5,
#               n_estimators=1000, objective='multi:softprob',
#               single_precision_histogram=True, tree_method='gpu_hist')

prediction = xgb_model.predict(Xdev)
print(classification_report(ydev, prediction)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.80      0.67      0.73      4236
     NEUTRAL       0.82      0.93      0.87      7593
    POSITIVE       0.82      0.73      0.78      3440

    accuracy                           0.81     15269
   macro avg       0.81      0.78      0.79     15269
weighted avg       0.81      0.81      0.81     15269



In [ ]:
prediction = xgb_model.predict(Xdev)
print(classification_report(ydev, prediction)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.79      0.68      0.73      4236
     NEUTRAL       0.82      0.92      0.87      7593
    POSITIVE       0.82      0.73      0.78      3440

    accuracy                           0.81     15269
   macro avg       0.81      0.78      0.79     15269
weighted avg       0.81      0.81      0.81     15269



In [ ]:
prediction = xgb_model.predict(Xdev)
print(classification_report(ydev, prediction)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.77      0.68      0.72      4236
     NEUTRAL       0.83      0.90      0.86      7593
    POSITIVE       0.79      0.74      0.77      3440

    accuracy                           0.80     15269
   macro avg       0.79      0.77      0.78     15269
weighted avg       0.80      0.80      0.80     15269



In [ ]:
xgb_model.save_model("./models/finaldata_xgboost_model.json")

In [ ]:
print(classification_report(prediction, y_count_val)) #Count Vectorizer

In [ ]:
print(classification_report(prediction, yval)) #Fasttext

In [ ]:
print(classification_report(prediction, yval)) #Sample weight

              precision    recall  f1-score   support

    NEGATIVE       0.66      0.78      0.72      5028
     NEUTRAL       0.92      0.81      0.86     12046
    POSITIVE       0.72      0.81      0.76      4331

    accuracy                           0.80     21405
   macro avg       0.77      0.80      0.78     21405
weighted avg       0.82      0.80      0.81     21405



In [ ]:
print(classification_report(prediction, yval)) #Sample weight

In [ ]:
from pprint import pprint

In [ ]:
def randomsearch():
  n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 15)]
  # Number of features to consider at every split
  max_features = ['auto', 'sqrt']
  # Maximum number of levels in tree
  max_depth = [int(x) for x in np.linspace(10, 200, num = 10)]
  max_depth.append(None)
  # Minimum number of samples required to split a node
  min_samples_split = [2, 4, 8, 16,32]
  # Minimum number of samples required at each leaf node
  min_samples_leaf = [1, 2, 4,8,16]
  # Method of selecting samples for training each tree
  bootstrap = [True, False]# Create the random grid
  random_grid = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'bootstrap': bootstrap}
                
  pprint(random_grid,compact=True)
  
  return random_grid

_ =randomsearch()

In [ ]:
import pickle
pickle.dump(xgb_model, open("/content/models/finaldata_xgboost_model_undersampled.pkl", "wb"))

import joblib
joblib.dump(xgb_model, "/content/models/finaldata_xgboost_model_undersampled.joblib") 

['/content/models/finaldata_xgboost_model_undersampled.joblib']

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_grid = randomsearch()


rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=4, random_state=2022, n_jobs = -1)# Fit the random search model
rf_random.fit(Xtrain, ytrain)


best_random = rf_random.best_estimator_

print("Paramters:")
pprint(rf_random.best_params_)

print("Training")
print(classification_report(ytrain,best_random.predict(Xtrain)))

print()

print("Testing")
print(classification_report(yval,best_random.predict(Xval)))

In [ ]:
 rf = RandomForestClassifier(max_features='log2', max_depth=7, class_weight='balanced', random_state=2022, n_jobs=-1)
 rf.fit(Xtrain, ytrain)

RandomForestClassifier(class_weight='balanced', max_features='log2', n_jobs=-1,
                       random_state=2022)

In [ ]:
predictions = rf.predict(Xdev)
print(classification_report(ydev, predictions)) #Random forest

              precision    recall  f1-score   support

    NEGATIVE       0.75      0.51      0.61      4236
     NEUTRAL       0.75      0.96      0.84      7593
    POSITIVE       0.81      0.63      0.71      3440

    accuracy                           0.76     15269
   macro avg       0.77      0.70      0.72     15269
weighted avg       0.76      0.76      0.75     15269



In [ ]:
import joblib
joblib.dump(rf, "./models/finaldata_random_forest.joblib", compress=3)

['./models/finaldata_random_forest.joblib']

In [ ]:
params = {
    'n_estimators':[500],
    'min_child_weight':[4,5], 
    'gamma':[i/10.0 for i in range(3,6)],  
    'subsample':[i/10.0 for i in range(6,11)],
    'colsample_bytree':[i/10.0 for i in range(6,11)], 
    'max_depth': [2,3,4,6,7],
    'objective': ['reg:squarederror', 'reg:tweedie'],
    'booster': ['gbtree', 'gblinear'],
    'eval_metric': ['rmse'],
    'eta': [i/10.0 for i in range(3,6)],
}

reg = XGBClassifier(tree_method = "gpu_hist", single_precision_histogram=True)

# run randomized search
n_iter_search = 100
random_search = RandomizedSearchCV(reg, param_distributions=params, random_state = 2022,
                                   n_iter=n_iter_search, cv=3, scoring='neg_mean_squared_error')



In [ ]:
random_search.fit(Xtrain, ytrain, sample_weight=classes_weights)

## Random Forest - Undersampling

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random_grid = randomsearch()


rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=4, random_state=2022, n_jobs = -1)# Fit the random search model
rf_random.fit(Xtrain, ytrain)


best_random = rf_random.best_estimator_

print("Paramters:")
pprint(rf_random.best_params_)

print("Training")
print(classification_report(ytrain,best_random.predict(Xtrain)))

print()

print("Testing")
print(classification_report(yval,best_random.predict(Xval)))

In [ ]:
 rf = RandomForestClassifier(max_features='sqrt', max_depth=9, class_weight='balanced', random_state=2022, n_jobs=-1)
 rf.fit(Xtrain, ytrain)

RandomForestClassifier(class_weight='balanced', max_depth=9,
                       max_features='sqrt', n_jobs=-1, random_state=2022)

In [ ]:
# RandomForestClassifier(class_weight='balanced', max_depth=7,
#                        max_features='log2', n_jobs=-1, random_state=2022)

predictions = rf.predict(Xdev)
print(classification_report(ydev, predictions)) #Random forest

              precision    recall  f1-score   support

    NEGATIVE       0.74      0.36      0.49      4236
     NEUTRAL       0.74      0.94      0.83      7593
    POSITIVE       0.63      0.66      0.65      3440

    accuracy                           0.71     15269
   macro avg       0.70      0.65      0.65     15269
weighted avg       0.72      0.71      0.69     15269



In [ ]:
# RandomForestClassifier(class_weight='balanced', max_depth=7,
#                        max_features='sqrt', n_jobs=-1, random_state=2022)

predictions = rf.predict(Xdev)
print(classification_report(ydev, predictions)) #Random forest

              precision    recall  f1-score   support

    NEGATIVE       0.72      0.40      0.52      4236
     NEUTRAL       0.74      0.93      0.83      7593
    POSITIVE       0.65      0.64      0.64      3440

    accuracy                           0.72     15269
   macro avg       0.70      0.66      0.66     15269
weighted avg       0.72      0.72      0.70     15269



In [ ]:
# RandomForestClassifier(class_weight='balanced', max_depth=9,
#                        max_features='sqrt', n_jobs=-1, random_state=2022)

predictions = rf.predict(Xdev)
print(classification_report(ydev, predictions)) #Random forest

              precision    recall  f1-score   support

    NEGATIVE       0.74      0.44      0.55      4236
     NEUTRAL       0.75      0.94      0.83      7593
    POSITIVE       0.68      0.65      0.67      3440

    accuracy                           0.74     15269
   macro avg       0.72      0.68      0.68     15269
weighted avg       0.73      0.74      0.72     15269



In [ ]:
import joblib
joblib.dump(rf, "./models/finaldata_random_forest_undersampled.joblib", compress=3)

['./models/finaldata_random_forest_undersampled.joblib']

In [ ]:
params = {
    'n_estimators':[500],
    'min_child_weight':[4,5], 
    'gamma':[i/10.0 for i in range(3,6)],  
    'subsample':[i/10.0 for i in range(6,11)],
    'colsample_bytree':[i/10.0 for i in range(6,11)], 
    'max_depth': [2,3,4,6,7],
    'objective': ['reg:squarederror', 'reg:tweedie'],
    'booster': ['gbtree', 'gblinear'],
    'eval_metric': ['rmse'],
    'eta': [i/10.0 for i in range(3,6)],
}

reg = XGBClassifier(tree_method = "gpu_hist", single_precision_histogram=True)

# run randomized search
n_iter_search = 100
random_search = RandomizedSearchCV(reg, param_distributions=params, random_state = 2022,
                                   n_iter=n_iter_search, cv=3, scoring='neg_mean_squared_error')



In [ ]:
random_search.fit(Xtrain, ytrain, sample_weight=classes_weights)

## Test data

In [ ]:
Xtest, ytest = prepare_data("./content/pickled_data/test.pkl")
testdf = pd.read_csv("test.csv")

In [ ]:
testdf = pickle.load(open("./content/pickled_data/test.pkl","rb"))
testdf

,label,tweet,cleaned_tweets,sent_vec
0,NEUTRAL,Effective but too-tepid biopic,effective but too tepid biopic,"[0.005347706, 0.03154791, 0.07578029, 0.006786..."
1,POSITIVE,If you sometimes like to go to the movies to h...,if you sometimes like to go to the movies to h...,"[0.0059573315, -0.03540754, 0.08914839, 0.0086..."
2,POSITIVE,"Emerges as something rare , an issue movie tha...",emerges as something rare an issue movie that ...,"[-0.009896653, 0.050302777, 0.09962972, 0.0017..."
3,NEUTRAL,The film provides some great insight into the ...,the film provides some great insight into the ...,"[0.00042353253, -0.02886739, 0.10362607, 0.020..."
4,POSITIVE,Offers that rare combination of entertainment ...,offers that rare combination of entertainment ...,"[-0.013658461, 0.021052698, 0.112356134, 0.049..."
...,...,...,...,...
30545,NEGATIVE,Another Day - this movie requires you to watch...,another day this movie requires you to watch i...,"[0.0075560594, -0.020790264, 0.080126695, 0.01..."
30546,POSITIVE,Why aren't more films (especially American) mo...,why are not more films (especially american) m...,"[0.002242105, -0.0093604475, 0.08441061, 0.006..."
30547,POSITIVE,Any film in the early days of Orson Welles is ...,any film in the early days of orson welles is ...,"[0.0020410304, -0.0238668, 0.09020886, 0.01618..."
30548,NEGATIVE,...and it is this film. I imagine that if inde...,and it is this film i imagine that if indeed t...,"[0.003267039, 0.00955246, 0.08873839, 0.007836..."


In [ ]:
pd.Series(ytest).value_counts()

NEUTRAL     15119
NEGATIVE     8527
POSITIVE     6904
dtype: int64

In [ ]:
Xtest.shape, ytest.shape

((30550, 300), (30550,))

## Benchmark dataset

In [ ]:
benchmark_df = pd.read_excel("Sentiment models.xlsx", engine='openpyxl')
benchmark_df

,Tweets,BERTweet,BERTweet_clean,bert-base-multilingual-uncased-sentiment,bert-base-multilingual-uncased-sentiment_clean,twitter-roberta,twitter-roberta_clean,VADER,VADER_clean
0,#GTvRR फायनल लढतीबाबत भाजपच्या नेत्याचा गंभीर ...,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
1,"Umran Malik, Hardik Pandya among XI in Harbhaj...",NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
2,दो साल से अर्जुन तेंदुलकर को मौका नहीं दे रही ...,NEUTRAL,NEUTRAL,NEGATIVE,NEGATIVE,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
3,TV 9 Special Report: सुब्रमण्यम स्वामी म्हणतात...,NEUTRAL,NEUTRAL,NEGATIVE,NEGATIVE,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
4,"#IPL मतलब इंडियन पैसा लीग जीतने पर करोड़ों, हा...",NEUTRAL,NEUTRAL,NEGATIVE,NEGATIVE,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
...,...,...,...,...,...,...,...,...,...
497,IPL 2022: Prashant Solanki reveals how MS Dhon...,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
498,David Warner was trolled by his fans for his c...,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,NEGATIVE,NEUTRAL,NEUTRAL,NEUTRAL
499,WATCH: Hardik Pandya consoles his wife Natasa ...,NEUTRAL,NEUTRAL,NEGATIVE,POSITIVE,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
500,@humpty_dvn @kaustats Most dot balls in an IPL...,NEUTRAL,NEUTRAL,NEGATIVE,POSITIVE,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL


In [ ]:
benchmark_df = benchmark_df[['Tweets']]

In [ ]:
benchmark_df['cleaned_tweets'] = benchmark_df.Tweets.apply(tweet_cleaning_for_sentiment_analysis)
benchmark_df['sent_vec'] = benchmark_df.cleaned_tweets.apply(fasttext_emb.get_sentence_vector)

In [ ]:
benchmark_df

In [ ]:
X = np.array(list(benchmark_df.sent_vec.apply(lambda x: list(x)).values))
benchmark_df['Random Forest'] = rf.predict(X)

In [ ]:
benchmark_df['Random Forest'] = benchmark_df['Random Forest'].replace(0, 'Neutral')
benchmark_df['Random Forest'] = benchmark_df['Random Forest'].replace(-1, 'Negative')
benchmark_df['Random Forest'] = benchmark_df['Random Forest'].replace(1, 'Positive')

benchmark_df

In [ ]:
benchmark_df['XGBoost'] = xgb_model.predict(X) 
benchmark_df

In [ ]:
benchmark_df['XGBoost'] = benchmark_df['XGBoost'].replace(0, 'Neutral')
benchmark_df['XGBoost'] = benchmark_df['XGBoost'].replace(-1, 'Negative')
benchmark_df['XGBoost'] = benchmark_df['XGBoost'].replace(1, 'Positive')

benchmark_df

In [ ]:
benchmark_df.to_excel("Sentiment.xlsx", index=False)

# Twitter custom test dataset

## Cleaning data

In [ ]:
import fasttext
import sys
import os
import nltk
nltk.download('punkt')

import pandas as pd
import datetime

from bs4 import BeautifulSoup

import re
import itertools
import emoji

import numpy as np
import pickle

import fasttext
import fasttext.util

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }

def strip_accents(text):
    if 'ø' in text or  'Ø' in text:
        #Do nothing when finding ø 
        return text   
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def tweet_cleaning_for_sentiment_analysis(tweet):    
    tweet = re.sub(r"(http?\://|https?\://|www|pic.)\S+", "", tweet) #Remove http links
    #Escaping HTML characters
    tweet = BeautifulSoup(tweet).get_text()
    #Special case not handled previously.
    tweet = tweet.replace('\x92',"'")
    #Removal of hastags/account
    #tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())
    tweet = tweet.replace("#", "").replace("_", " ")
    #Removal of address
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    #Removal of Punctuation
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    #Lower case
    tweet = tweet.lower()
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)", "USER", tweet).split())
    #CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS = load_dict_contractions()
    tweet = tweet.replace("’","'")
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    # Standardizing words
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    #Deal with smileys
    #source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    #Deal with emojis
    #tweet = emoji.demojize(tweet)
    #Strip accents
    #tweet= strip_accents(tweet)
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())
    
    # DO NOT REMOVE STOP WORDS FOR SENTIMENT ANALYSIS - OR AT LEAST NOT NEGATIVE ONES

    return tweet

In [ ]:
tweet_cleaning_for_sentiment_analysis(data.iloc[29, :]['tweet'])

'melusi 4k are you shocked USER USER USER USER USER rainbowsixsiege'

## Word embeddings

### Download embedddings

In [ ]:
!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
!unzip crawl-300d-2M-subword.zip
%cd /content

/content/fasttext-embeddings
--2022-07-21 14:55:23--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5828358084 (5.4G) [application/zip]
Saving to: ‘crawl-300d-2M-subword.zip’

crawl-300d-2M-subwo 100%[===================>]   5.43G  32.1MB/s    in 3m 13s  

2022-07-21 14:58:37 (28.8 MB/s) - ‘crawl-300d-2M-subword.zip’ saved [5828358084/5828358084]

Archive:  crawl-300d-2M-subword.zip
  inflating: crawl-300d-2M-subword.vec  
  inflating: crawl-300d-2M-subword.bin  
/content


In [ ]:
!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.vec.zip
!unzip wiki-news-300d-1M-subword.vec.zip
%cd /content

mkdir: cannot create directory ‘fasttext-embeddings’: File exists
/content/fasttext-embeddings
--2022-07-21 15:24:21--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 587879973 (561M) [application/zip]
Saving to: ‘wiki-news-300d-1M-subword.vec.zip’

wiki-news-300d-1M-s 100%[===================>] 560.65M  36.8MB/s    in 16s     

2022-07-21 15:24:38 (34.7 MB/s) - ‘wiki-news-300d-1M-subword.vec.zip’ saved [587879973/587879973]

Archive:  wiki-news-300d-1M-subword.vec.zip
  inflating: wiki-news-300d-1M-subword.vec  
/content


In [ ]:
!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gzip -d cc.en.300.bin.gz
%cd /content

mkdir: cannot create directory ‘fasttext-embeddings’: File exists
/content/fasttext-embeddings
--2022-07-21 15:28:31--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  28.1MB/s    in 2m 18s  

2022-07-21 15:30:49 (31.2 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]

/content


In [ ]:
# !mkdir fasttext-embeddings
# %cd fasttext-embeddings
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
# !unzip crawl-300d-2M.vec.zip

In [ ]:
%cd /content

/content


### Loading model

In [ ]:

# fasttext_emb = fasttext.load_model('/content/fasttext-embeddings/cc.en.300.bin')
# print(ft.get_dimension())

# fasttext.util.reduce_model(ft, 100)
# print(ft.get_dimension())

NameError: ignored

In [ ]:
fasttext_emb = fasttext.FastText.load_model('./fasttext-embeddings/crawl-300d-2M-subword.bin')
#fasttext_emb = fasttext.FastText.load_model('./fasttext-embeddings/wiki-news-300d-1M-subword.vec')

In [ ]:
fasttext_emb.get_nearest_neighbors("__user__")

[(0.6122851371765137, 'intrebari'),
 (0.6034965515136719, 'vmware-modconfig'),
 (0.5972209572792053, 'eclipselink-orm.xml'),
 (0.5965851545333862, 'base-config'),
 (0.5962321162223816, 'hostnamectl'),
 (0.5956684350967407, 'myconfig'),
 (0.593761146068573, 'webpack.config.js'),
 (0.5922401547431946, 'mkvirtualenv'),
 (0.5871965289115906, 'libxml2-dev'),
 (0.5849608182907104, 'jboss-service.xml')]

## Preparing data

In [ ]:
data = pd.read_csv("Sentiment models - Labelled Test Data.csv")

data['cleaned_tweets'] = data.tweet.astype(str).apply(tweet_cleaning_for_sentiment_analysis)
data['sent_vec'] = data.cleaned_tweets.apply(fasttext_emb.get_sentence_vector)

with open("twitter-test-data.pkl", "wb") as f:
  pickle.dump(data, f)

del data

In [ ]:
def prepare_data(pickle_filepath):
  data = pickle.load(open(pickle_filepath,'rb'))
  
  X = np.array(list(data.sent_vec.apply(lambda x: list(x)).values))
  
  return X, data.label.values

In [ ]:
Xtest, ytest = prepare_data("twitter-test-data.pkl")

In [ ]:
Xtest.shape, ytest.shape

((62, 300), (62,))

## Test models

In [ ]:
!gdown https://drive.google.com/file/d/1z6wJGTVLsWQW8lhNGaI1rogqxiAQP2YH/view?usp=sharing --fuzzy

Downloading...
From: https://drive.google.com/uc?id=1z6wJGTVLsWQW8lhNGaI1rogqxiAQP2YH
To: /content/models.rar
100% 83.2M/83.2M [00:03<00:00, 21.3MB/s]


In [ ]:
!unrar e models.rar /content/models


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from models.rar


Would you like to replace the existing file /content/models/finaldata_random_forest.joblib
26214400 bytes, modified on 2022-07-28 10:50
with a new one
63981274 bytes, modified on 2022-07-21 17:31

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit A

Extracting  /content/models/finaldata_random_forest.joblib                 5% 10% 15% 20% 25% 30% 35% 40% 45% 50% 55% 60% 65% 70% 75%  OK 
Extracting  /content/models/finaldata_random_forest_undersampled.joblib      76% 77% 78% 79% 80%  OK 
Extracting  /content/models/finaldata_xgboost_model.json                  85% 88%  OK 
Extracting  /content/models/finaldata_xgboost_model_undersampled.json      93% 98% 99%  OK 
All OK


In [ ]:
import joblib
from sklearn.metrics import classification_report

### Random Forest

In [ ]:
rf = joblib.load("/content/models/finaldata_random_forest.joblib")

In [ ]:
predictions = rf.predict(Xtest)
print(classification_report(ytest, predictions)) #Random forest

              precision    recall  f1-score   support

    NEGATIVE       0.25      0.15      0.19        20
     NEUTRAL       0.20      0.64      0.31        14
    POSITIVE       0.50      0.11      0.18        28

    accuracy                           0.24        62
   macro avg       0.32      0.30      0.22        62
weighted avg       0.35      0.24      0.21        62



In [ ]:
testdf = pd.DataFrame()

In [ ]:
testdf['Random Forest'] = predictions

### Random Forest - Undersampled

In [ ]:
rf = joblib.load("/content/models/finaldata_random_forest_undersampled.joblib")

In [ ]:
predictions = rf.predict(Xtest)
print(classification_report(ytest, predictions)) #Random forest

              precision    recall  f1-score   support

    NEGATIVE       0.38      0.40      0.39        20
     NEUTRAL       0.21      0.36      0.26        14
    POSITIVE       0.65      0.39      0.49        28

    accuracy                           0.39        62
   macro avg       0.41      0.38      0.38        62
weighted avg       0.46      0.39      0.41        62



In [ ]:
testdf['Random Forest - Undersampled'] = predictions

### XGBoost

In [ ]:
predictions = xgb_model.predict(Xtest)
print(classification_report(ytest, predictions)) #xgboost

              precision    recall  f1-score   support

    NEGATIVE       0.41      0.35      0.38        20
     NEUTRAL       0.17      0.29      0.22        14
    POSITIVE       0.59      0.46      0.52        28

    accuracy                           0.39        62
   macro avg       0.39      0.37      0.37        62
weighted avg       0.44      0.39      0.41        62



In [ ]:
testdf['XGBoost'] = predictions

### XGBoost - Undersampled

In [ ]:
predictions = xgb_model.predict(Xtest)
print(classification_report(ytest, predictions)) #xgboost

              precision    recall  f1-score   support

    NEGATIVE       0.48      0.55      0.51        20
     NEUTRAL       0.36      0.36      0.36        14
    POSITIVE       0.68      0.61      0.64        28

    accuracy                           0.53        62
   macro avg       0.51      0.50      0.50        62
weighted avg       0.54      0.53      0.54        62



In [ ]:
testdf['XGBoost - Undersampled'] = predictions

In [ ]:
testdf

,Random Forest,Random Forest - Undersampled,XGBoost,XGBoost - Undersampled
0,NEUTRAL,POSITIVE,POSITIVE,POSITIVE
1,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
2,NEUTRAL,POSITIVE,POSITIVE,POSITIVE
3,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
4,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE
...,...,...,...,...
57,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE
58,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE
59,NEUTRAL,POSITIVE,POSITIVE,POSITIVE
60,NEUTRAL,NEGATIVE,NEUTRAL,NEGATIVE


### BERTweet

In [ ]:
data = pickle.load(open("/content/twitter-test-data.pkl", "rb"))
data

,tweet,label,cleaned_tweets,sent_vec
0,‘It has been a really memorable season’ – Jos ...,POSITIVE,‘it has been a really memorable season' – jos ...,"[0.0044800234, -0.024687007, 0.03165525, 0.026..."
1,"Mumbai Indians and Titans Cricket batter, Dewa...",POSITIVE,mumbai indians and titans cricket batter dewal...,"[-0.013870388, -0.042408988, 0.0869929, 0.0214..."
2,There were several uncapped players who proved...,POSITIVE,there were several uncapped players who proved...,"[-0.012238055, -0.05264265, 0.08353262, 0.0121..."
3,AFTER ARGUING WITH VERIZON REGARDING THE DROPP...,NEGATIVE,after arguing with verizon regarding the dropp...,"[0.0059597986, -0.049668398, 0.0746304, 0.0094..."
4,A usable keyboard actually turns a PDA into a ...,POSITIVE,a usable keyboard actually turns a pda into a ...,"[-0.0028914495, 0.009457815, 0.075295255, 0.00..."
...,...,...,...,...
57,@satyanadella @Microsoft thanks for celebratin...,POSITIVE,USER USER thanks for celebrating diversity we ...,"[-0.0034089629, -0.049936995, 0.08740346, 0.02..."
58,Thatâ€™s the best one wtf?,POSITIVE,thatâ€™s the best one wtf,"[-0.00953177, -0.004613374, 0.10661884, -0.008..."
59,AOC you make some very ignorant comments\n Whe...,NEUTRAL,aoc you make some very ignorant comments when ...,"[-0.0043282043, -0.063566, 0.087194875, 0.0080..."
60,oooooh shit i think my motherboard is already ...,POSITIVE,ooh shit i think my motherboard is already com...,"[-0.0018573017, 0.026468603, 0.07860464, 0.002..."


In [ ]:
testdf['BERTweet'] = data['cleaned_tweets'].apply(sa1.inference)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print(classification_report(ytest, testdf['BERTweet'].values)) #BERtweet

              precision    recall  f1-score   support

    NEGATIVE       0.48      0.60      0.53        20
     NEUTRAL       0.23      0.21      0.22        14
    POSITIVE       0.83      0.71      0.77        28

    accuracy                           0.56        62
   macro avg       0.51      0.51      0.51        62
weighted avg       0.58      0.56      0.57        62



### BERT base

In [ ]:
testdf['bert-base-multilingual-uncased-sentiment_clean'] = data['cleaned_tweets'].apply(sa2.inference)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print(classification_report(ytest, testdf['bert-base-multilingual-uncased-sentiment_clean'].values)) #BERT Base

              precision    recall  f1-score   support

    NEGATIVE       0.50      0.70      0.58        20
     NEUTRAL       1.00      0.07      0.13        14
    POSITIVE       0.70      0.82      0.75        28

    accuracy                           0.61        62
   macro avg       0.73      0.53      0.49        62
weighted avg       0.70      0.61      0.56        62



In [ ]:
testdf['tweet'] = data['tweet']
testdf['label'] = data['label']

testdf['cleaned_tweets'] = data['cleaned_tweets']

,Random Forest,Random Forest - Undersampled,XGBoost,XGBoost - Undersampled,BERTweet,bert-base-multilingual-uncased-sentiment_clean,tweet,label
0,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,POSITIVE,POSITIVE,‘It has been a really memorable season’ – Jos ...,POSITIVE
1,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,"Mumbai Indians and Titans Cricket batter, Dewa...",POSITIVE
2,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,POSITIVE,POSITIVE,There were several uncapped players who proved...,POSITIVE
3,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEGATIVE,NEGATIVE,AFTER ARGUING WITH VERIZON REGARDING THE DROPP...,NEGATIVE
4,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE,POSITIVE,POSITIVE,A usable keyboard actually turns a PDA into a ...,POSITIVE
...,...,...,...,...,...,...,...,...
57,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,@satyanadella @Microsoft thanks for celebratin...,POSITIVE
58,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,POSITIVE,NEGATIVE,Thatâ€™s the best one wtf?,POSITIVE
59,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,NEGATIVE,NEGATIVE,AOC you make some very ignorant comments\n Whe...,NEUTRAL
60,NEUTRAL,NEGATIVE,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE,oooooh shit i think my motherboard is already ...,POSITIVE


In [ ]:
testdf

,Random Forest,Random Forest - Undersampled,XGBoost,XGBoost - Undersampled,BERTweet,bert-base-multilingual-uncased-sentiment_clean,tweet,label,cleaned_tweets
0,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,POSITIVE,POSITIVE,‘It has been a really memorable season’ – Jos ...,POSITIVE,‘it has been a really memorable season' – jos ...
1,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,"Mumbai Indians and Titans Cricket batter, Dewa...",POSITIVE,mumbai indians and titans cricket batter dewal...
2,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,POSITIVE,POSITIVE,There were several uncapped players who proved...,POSITIVE,there were several uncapped players who proved...
3,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEGATIVE,NEGATIVE,AFTER ARGUING WITH VERIZON REGARDING THE DROPP...,NEGATIVE,after arguing with verizon regarding the dropp...
4,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE,POSITIVE,POSITIVE,A usable keyboard actually turns a PDA into a ...,POSITIVE,a usable keyboard actually turns a pda into a ...
...,...,...,...,...,...,...,...,...,...
57,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,@satyanadella @Microsoft thanks for celebratin...,POSITIVE,USER USER thanks for celebrating diversity we ...
58,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,POSITIVE,NEGATIVE,Thatâ€™s the best one wtf?,POSITIVE,thatâ€™s the best one wtf
59,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,NEGATIVE,NEGATIVE,AOC you make some very ignorant comments\n Whe...,NEUTRAL,aoc you make some very ignorant comments when ...
60,NEUTRAL,NEGATIVE,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE,oooooh shit i think my motherboard is already ...,POSITIVE,ooh shit i think my motherboard is already com...


In [ ]:
testdf.to_excel("twitter-sample-data-results.xlsx")

In [ ]:
with open("./test-predictions-undersampled.pkl","wb") as f:
  pickle.dump(testdf, f)

# Twitter custom test dataset - Without emojis

In [ ]:
!pip install fasttext emoji -q

     |████████████████████████████████| 68 kB 5.1 MB/s 
     |████████████████████████████████| 197 kB 34.0 MB/s 


## Cleaning data

In [ ]:
import sys
import os
import nltk
nltk.download('punkt')

import pandas as pd
import datetime

from bs4 import BeautifulSoup

import re
import itertools
import emoji

import numpy as np
import pickle

import fasttext
import fasttext.util

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }

def strip_accents(text):
    if 'ø' in text or  'Ø' in text:
        #Do nothing when finding ø 
        return text   
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def tweet_cleaning_for_sentiment_analysis(tweet):    
    tweet = re.sub(r"(http?\://|https?\://|www|pic.)\S+", "", tweet) #Remove http links
    #Escaping HTML characters
    tweet = BeautifulSoup(tweet).get_text()
    #Special case not handled previously.
    tweet = tweet.replace('\x92',"'")
    #Removal of hastags/account
    #tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())
    tweet = tweet.replace("#", "").replace("_", " ")
    #Removal of address
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    #Removal of Punctuation
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    #Lower case
    tweet = tweet.lower()
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)", "USER", tweet).split())
    #CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS = load_dict_contractions()
    tweet = tweet.replace("’","'")
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    # Standardizing words
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    #Deal with smileys
    #source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    #Deal with emojis
    tweet = emoji.demojize(tweet)
    #Strip accents
    #tweet= strip_accents(tweet)
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())
    
    # DO NOT REMOVE STOP WORDS FOR SENTIMENT ANALYSIS - OR AT LEAST NOT NEGATIVE ONES

    return tweet

In [ ]:
tweet_cleaning_for_sentiment_analysis(data.iloc[6, :]['tweet'])

'our kaptaan sahaab has scored 550+ runs in five consecutive seasons smiling_face_with_sunglasses abapnibaarihai flexed_biceps ipl2022 trophy bhaukaalmachadenge lsg lucknowsupergiants t20 tataipl lucknow uttarpradesh lsg2022'

## Word embeddings

### Download embedddings

In [ ]:
!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
!unzip crawl-300d-2M-subword.zip
%cd /content

/content/fasttext-embeddings
--2022-08-01 08:21:38--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5828358084 (5.4G) [application/zip]
Saving to: ‘crawl-300d-2M-subword.zip’

crawl-300d-2M-subwo 100%[===================>]   5.43G  12.5MB/s    in 8m 40s  

2022-08-01 08:30:20 (10.7 MB/s) - ‘crawl-300d-2M-subword.zip’ saved [5828358084/5828358084]

Archive:  crawl-300d-2M-subword.zip
  inflating: crawl-300d-2M-subword.vec  
  inflating: crawl-300d-2M-subword.bin  
/content


In [ ]:
!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.vec.zip
!unzip wiki-news-300d-1M-subword.vec.zip
%cd /content

mkdir: cannot create directory ‘fasttext-embeddings’: File exists
/content/fasttext-embeddings
--2022-07-21 15:24:21--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 587879973 (561M) [application/zip]
Saving to: ‘wiki-news-300d-1M-subword.vec.zip’

wiki-news-300d-1M-s 100%[===================>] 560.65M  36.8MB/s    in 16s     

2022-07-21 15:24:38 (34.7 MB/s) - ‘wiki-news-300d-1M-subword.vec.zip’ saved [587879973/587879973]

Archive:  wiki-news-300d-1M-subword.vec.zip
  inflating: wiki-news-300d-1M-subword.vec  
/content


In [ ]:
!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gzip -d cc.en.300.bin.gz
%cd /content

mkdir: cannot create directory ‘fasttext-embeddings’: File exists
/content/fasttext-embeddings
--2022-07-21 15:28:31--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  28.1MB/s    in 2m 18s  

2022-07-21 15:30:49 (31.2 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]

/content


In [ ]:
# !mkdir fasttext-embeddings
# %cd fasttext-embeddings
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
# !unzip crawl-300d-2M.vec.zip

In [ ]:
%cd /content

/content


### Loading model

In [ ]:

# fasttext_emb = fasttext.load_model('/content/fasttext-embeddings/cc.en.300.bin')
# print(ft.get_dimension())

# fasttext.util.reduce_model(ft, 100)
# print(ft.get_dimension())

NameError: ignored

In [ ]:
fasttext_emb = fasttext.FastText.load_model('./fasttext-embeddings/crawl-300d-2M-subword.bin')
#fasttext_emb = fasttext.FastText.load_model('./fasttext-embeddings/wiki-news-300d-1M-subword.vec')

In [ ]:
fasttext_emb.get_nearest_neighbors("__user__")

[(0.6122851371765137, 'intrebari'),
 (0.6034965515136719, 'vmware-modconfig'),
 (0.5972209572792053, 'eclipselink-orm.xml'),
 (0.5965851545333862, 'base-config'),
 (0.5962321162223816, 'hostnamectl'),
 (0.5956684350967407, 'myconfig'),
 (0.593761146068573, 'webpack.config.js'),
 (0.5922401547431946, 'mkvirtualenv'),
 (0.5871965289115906, 'libxml2-dev'),
 (0.5849608182907104, 'jboss-service.xml')]

## Preparing data

In [ ]:
data = pd.read_csv("Sentiment models - Labelled Test Data.csv")

data['cleaned_tweets'] = data.tweet.astype(str).apply(tweet_cleaning_for_sentiment_analysis)
data['sent_vec'] = data.cleaned_tweets.apply(fasttext_emb.get_sentence_vector)

with open("twitter-test-data-without-emojis.pkl", "wb") as f:
  pickle.dump(data, f)

# del data

In [ ]:
def prepare_data(pickle_filepath):
  data = pickle.load(open(pickle_filepath,'rb'))
  
  X = np.array(list(data.sent_vec.apply(lambda x: list(x)).values))
  
  return X, data.label.values

In [ ]:
Xtest, ytest = prepare_data("twitter-test-data-without-emojis.pkl")

In [ ]:
Xtest.shape, ytest.shape

((62, 300), (62,))

In [ ]:
del fasttext_emb

## Test models

In [ ]:
!gdown https://drive.google.com/file/d/16RTmAYGJ-FWRJ1R_oYcNE-3opdopb37X/view --fuzzy

Downloading...
From: https://drive.google.com/uc?id=16RTmAYGJ-FWRJ1R_oYcNE-3opdopb37X
To: /content/models.rar
100% 116M/116M [00:00<00:00, 200MB/s]


In [ ]:
!mkdir models
!unrar e models.rar /content/models


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from models.rar

Extracting  /content/models/finaldata_random_forest.joblib                 3%  7% 10% 14% 18% 21% 25% 28% 32% 36% 39% 43% 46% 50% 54%  OK 
Extracting  /content/models/finaldata_random_forest_undersampled.joblib      54% 55% 56% 57%  OK 
Extracting  /content/models/finaldata_xgboost_model.json                  60% 63%  OK 
Extracting  /content/models/finaldata_xgboost_model_undersampled.json      67% 70% 71%  OK 
Extracting  /content/models/finaldata_xgboost_model_undersampled.joblib      75% 78% 79%  OK 
Extracting  /content/models/finaldata_xgboost_model.pkl                   83% 85%  OK 
Extracting  /content/models/finaldata_xgboost_model.joblib                89% 91%  OK 
Extracting  /content/models/finaldata_xgboost_model_undersam

In [ ]:
import joblib
from sklearn.metrics import classification_report

### Random Forest

In [ ]:
rf = joblib.load("/content/models/finaldata_random_forest.joblib")

In [ ]:
predictions = rf.predict(Xtest)
print(classification_report(ytest, predictions)) #Random forest

              precision    recall  f1-score   support

    NEGATIVE       0.27      0.15      0.19        20
     NEUTRAL       0.20      0.64      0.31        14
    POSITIVE       0.50      0.11      0.18        28

    accuracy                           0.24        62
   macro avg       0.32      0.30      0.23        62
weighted avg       0.36      0.24      0.21        62



In [ ]:
testdf = pd.DataFrame()

In [ ]:
testdf['Random Forest'] = predictions

### Random Forest - Undersampled

In [ ]:
rf = joblib.load("/content/models/finaldata_random_forest_undersampled.joblib")

In [ ]:
predictions = rf.predict(Xtest)
print(classification_report(ytest, predictions)) #Random forest

              precision    recall  f1-score   support

    NEGATIVE       0.38      0.40      0.39        20
     NEUTRAL       0.21      0.36      0.26        14
    POSITIVE       0.65      0.39      0.49        28

    accuracy                           0.39        62
   macro avg       0.41      0.38      0.38        62
weighted avg       0.46      0.39      0.41        62



In [ ]:
testdf['Random Forest - Undersampled'] = predictions

### XGBoost

In [ ]:
xgb_model = joblib.load("/content/models/finaldata_xgboost_model.joblib")

In [ ]:
predictions = xgb_model.predict(Xtest)
print(classification_report(ytest, predictions)) #xgboost

              precision    recall  f1-score   support

    NEGATIVE       0.41      0.35      0.38        20
     NEUTRAL       0.17      0.29      0.22        14
    POSITIVE       0.59      0.46      0.52        28

    accuracy                           0.39        62
   macro avg       0.39      0.37      0.37        62
weighted avg       0.44      0.39      0.41        62



In [ ]:
testdf['XGBoost'] = predictions

### XGBoost - Undersampled

In [ ]:
xgb_model = joblib.load("/content/models/finaldata_xgboost_model_undersampled.joblib")

In [ ]:
predictions = xgb_model.predict(Xtest)
print(classification_report(ytest, predictions)) #xgboost

              precision    recall  f1-score   support

    NEGATIVE       0.48      0.55      0.51        20
     NEUTRAL       0.36      0.36      0.36        14
    POSITIVE       0.68      0.61      0.64        28

    accuracy                           0.53        62
   macro avg       0.51      0.50      0.50        62
weighted avg       0.54      0.53      0.54        62



In [ ]:
testdf['XGBoost - Undersampled'] = predictions

In [ ]:
testdf

,Random Forest,Random Forest - Undersampled,XGBoost,XGBoost - Undersampled
0,NEUTRAL,POSITIVE,POSITIVE,POSITIVE
1,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
2,NEUTRAL,POSITIVE,POSITIVE,POSITIVE
3,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
4,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE
...,...,...,...,...
57,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE
58,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE
59,NEUTRAL,POSITIVE,POSITIVE,POSITIVE
60,NEUTRAL,NEGATIVE,NEUTRAL,NEGATIVE


### BERTweet

In [ ]:
data = pickle.load(open("/content/twitter-test-data-without-emojis.pkl", "rb"))
data

,tweet,label,cleaned_tweets,sent_vec
0,‘It has been a really memorable season’ – Jos ...,POSITIVE,‘it has been a really memorable season' – jos ...,"[0.0044800234, -0.024687007, 0.03165525, 0.026..."
1,"Mumbai Indians and Titans Cricket batter, Dewa...",POSITIVE,mumbai indians and titans cricket batter dewal...,"[-0.013870388, -0.042408988, 0.0869929, 0.0214..."
2,There were several uncapped players who proved...,POSITIVE,there were several uncapped players who proved...,"[-0.012238055, -0.05264265, 0.08353262, 0.0121..."
3,AFTER ARGUING WITH VERIZON REGARDING THE DROPP...,NEGATIVE,after arguing with verizon regarding the dropp...,"[0.0059597986, -0.049668398, 0.0746304, 0.0094..."
4,A usable keyboard actually turns a PDA into a ...,POSITIVE,a usable keyboard actually turns a pda into a ...,"[-0.0028914495, 0.009457815, 0.075295255, 0.00..."
...,...,...,...,...
57,@satyanadella @Microsoft thanks for celebratin...,POSITIVE,USER USER thanks for celebrating diversity we ...,"[-0.0034089629, -0.049936995, 0.08740346, 0.02..."
58,Thatâ€™s the best one wtf?,POSITIVE,thatâ€ trade_mark s the best one wtf,"[0.010495955, -0.015235853, 0.10701324, 0.0062..."
59,AOC you make some very ignorant comments\n Whe...,NEUTRAL,aoc you make some very ignorant comments when ...,"[-0.0043282043, -0.063566, 0.087194875, 0.0080..."
60,oooooh shit i think my motherboard is already ...,POSITIVE,ooh shit i think my motherboard is already com...,"[-0.0018573017, 0.026468603, 0.07860464, 0.002..."


In [ ]:
testdf['BERTweet'] = data['cleaned_tweets'].apply(sa1.inference)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print(classification_report(ytest, testdf['BERTweet'].values)) #BERtweet

              precision    recall  f1-score   support

    NEGATIVE       0.50      0.60      0.55        20
     NEUTRAL       0.21      0.21      0.21        14
    POSITIVE       0.83      0.71      0.77        28

    accuracy                           0.56        62
   macro avg       0.52      0.51      0.51        62
weighted avg       0.59      0.56      0.57        62



### BERT base

In [ ]:
testdf['bert-base-multilingual-uncased-sentiment_clean'] = data['cleaned_tweets'].apply(sa2.inference)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print(classification_report(ytest, testdf['bert-base-multilingual-uncased-sentiment_clean'].values)) #BERT Base

              precision    recall  f1-score   support

    NEGATIVE       0.52      0.70      0.60        20
     NEUTRAL       1.00      0.07      0.13        14
    POSITIVE       0.71      0.86      0.77        28

    accuracy                           0.63        62
   macro avg       0.74      0.54      0.50        62
weighted avg       0.71      0.63      0.57        62



In [ ]:
testdf['tweet'] = data['tweet']
testdf['label'] = data['label']

testdf['cleaned_tweets'] = data['cleaned_tweets']

testdf

,Random Forest,Random Forest - Undersampled,XGBoost,XGBoost - Undersampled,BERTweet,bert-base-multilingual-uncased-sentiment_clean,tweet,label,cleaned_tweets
0,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,POSITIVE,POSITIVE,‘It has been a really memorable season’ – Jos ...,POSITIVE,‘it has been a really memorable season' – jos ...
1,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,"Mumbai Indians and Titans Cricket batter, Dewa...",POSITIVE,mumbai indians and titans cricket batter dewal...
2,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,POSITIVE,POSITIVE,There were several uncapped players who proved...,POSITIVE,there were several uncapped players who proved...
3,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEGATIVE,NEGATIVE,AFTER ARGUING WITH VERIZON REGARDING THE DROPP...,NEGATIVE,after arguing with verizon regarding the dropp...
4,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE,POSITIVE,POSITIVE,A usable keyboard actually turns a PDA into a ...,POSITIVE,a usable keyboard actually turns a pda into a ...
...,...,...,...,...,...,...,...,...,...
57,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,@satyanadella @Microsoft thanks for celebratin...,POSITIVE,USER USER thanks for celebrating diversity we ...
58,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE,POSITIVE,POSITIVE,Thatâ€™s the best one wtf?,POSITIVE,thatâ€ trade_mark s the best one wtf
59,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,NEGATIVE,NEGATIVE,AOC you make some very ignorant comments\n Whe...,NEUTRAL,aoc you make some very ignorant comments when ...
60,NEUTRAL,NEGATIVE,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE,oooooh shit i think my motherboard is already ...,POSITIVE,ooh shit i think my motherboard is already com...


In [ ]:
testdf.to_excel("twitter-sample-data-results-without-emojis.xlsx")

In [ ]:
with open("./test-predictions-undersampled.pkl","wb") as f:
  pickle.dump(testdf, f)

# Twitter custom test dataset - Part 2

In [ ]:
!pip install fasttext emoji -q

     |████████████████████████████████| 68 kB 3.4 MB/s 
     |████████████████████████████████| 197 kB 10.4 MB/s 


In [ ]:
import sys
import os
import nltk
nltk.download('punkt')

import pandas as pd
import datetime

from bs4 import BeautifulSoup

import re
import itertools
import emoji

import numpy as np
import pickle

import fasttext
import fasttext.util

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Cleaning data

In [ ]:
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }

def strip_accents(text):
    if 'ø' in text or  'Ø' in text:
        #Do nothing when finding ø 
        return text   
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def tweet_cleaning_for_sentiment_analysis(tweet):    
    tweet = re.sub(r"(http?\://|https?\://|www|pic.)\S+", "", tweet) #Remove http links
    #Escaping HTML characters
    tweet = BeautifulSoup(tweet).get_text()
    #Special case not handled previously.
    tweet = tweet.replace('\x92',"'")
    #Removal of hastags/account
    #tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", tweet).split())
    tweet = tweet.replace("#", "").replace("_", " ")
    #Removal of address
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    #Removal of Punctuation
    tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
    #Lower case
    tweet = tweet.lower()
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)", "USER", tweet).split())
    #CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS = load_dict_contractions()
    tweet = tweet.replace("’","'")
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    # Standardizing words
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    #Deal with smileys
    #source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = tweet.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    tweet = " ".join(reformed)
    #Deal with emojis
    # tweet = emoji.demojize(tweet)
    #Strip accents
    #tweet= strip_accents(tweet)
    tweet = tweet.replace(":"," ")
    tweet = ' '.join(tweet.split())
    
    # DO NOT REMOVE STOP WORDS FOR SENTIMENT ANALYSIS - OR AT LEAST NOT NEGATIVE ONES

    return tweet

In [ ]:
tweet_cleaning_for_sentiment_analysis(data.iloc[6, :]['tweet'])

'our kaptaan sahaab has scored 550+ runs in five consecutive seasons smiling_face_with_sunglasses abapnibaarihai flexed_biceps ipl2022 trophy bhaukaalmachadenge lsg lucknowsupergiants t20 tataipl lucknow uttarpradesh lsg2022'

## Word embeddings

### Download embedddings

In [ ]:
!mkdir fasttext-embeddings
%cd fasttext-embeddings
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
!unzip crawl-300d-2M-subword.zip
%cd /content

/content/fasttext-embeddings
--2022-08-02 11:46:58--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5828358084 (5.4G) [application/zip]
Saving to: ‘crawl-300d-2M-subword.zip’

crawl-300d-2M-subwo 100%[===================>]   5.43G  35.0MB/s    in 2m 22s  

2022-08-02 11:49:21 (39.1 MB/s) - ‘crawl-300d-2M-subword.zip’ saved [5828358084/5828358084]

Archive:  crawl-300d-2M-subword.zip
  inflating: crawl-300d-2M-subword.vec  
  inflating: crawl-300d-2M-subword.bin  
/content


In [ ]:
# !mkdir fasttext-embeddings
# %cd fasttext-embeddings
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M-subword.vec.zip
# !unzip wiki-news-300d-1M-subword.vec.zip
# %cd /content

In [ ]:
# !mkdir fasttext-embeddings
# %cd fasttext-embeddings
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
# !gzip -d cc.en.300.bin.gz
# %cd /content

In [ ]:
# !mkdir fasttext-embeddings
# %cd fasttext-embeddings
# !wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
# !unzip crawl-300d-2M.vec.zip

In [ ]:
%cd /content

/content


### Loading model

In [ ]:

# fasttext_emb = fasttext.load_model('/content/fasttext-embeddings/cc.en.300.bin')
# print(ft.get_dimension())

# fasttext.util.reduce_model(ft, 100)
# print(ft.get_dimension())

In [ ]:
fasttext_emb = fasttext.FastText.load_model('./fasttext-embeddings/crawl-300d-2M-subword.bin')
#fasttext_emb = fasttext.FastText.load_model('./fasttext-embeddings/wiki-news-300d-1M-subword.vec')

In [ ]:
fasttext_emb.get_nearest_neighbors("smiling_face_with_sunglasses")

[(0.6122851371765137, 'intrebari'),
 (0.6034965515136719, 'vmware-modconfig'),
 (0.5972209572792053, 'eclipselink-orm.xml'),
 (0.5965851545333862, 'base-config'),
 (0.5962321162223816, 'hostnamectl'),
 (0.5956684350967407, 'myconfig'),
 (0.593761146068573, 'webpack.config.js'),
 (0.5922401547431946, 'mkvirtualenv'),
 (0.5871965289115906, 'libxml2-dev'),
 (0.5849608182907104, 'jboss-service.xml')]

## Preparing data

In [ ]:
data = pd.read_csv("Sentiment models - New.csv")

data['cleaned_tweets'] = data.tweet.astype(str).apply(tweet_cleaning_for_sentiment_analysis)
data['sent_vec'] = data.cleaned_tweets.apply(fasttext_emb.get_sentence_vector)

with open("twitter-test-data-new.pkl", "wb") as f:
  pickle.dump(data, f)

del data

,label,tweet,cleaned_tweets,sent_vec
0,0,@user when a father is dysfunctional and is so...,USER when a father is dysfunctional and is so ...,"[-0.014220357, -0.008476666, 0.08933757, 0.003..."
1,0,@user @user thanks for #lyft credit i can't us...,USER USER thanks for lyft credit i cannot use ...,"[-0.0136345625, -0.04320838, 0.083086476, -0.0..."
2,0,bihday your majesty,bihday your majesty,"[-0.024211615, -0.04941867, 0.0942983, -0.0160..."
3,0,#model i love u take with u all the time in ur...,model i love u take with u all the time in urã...,"[-0.0042502587, -0.044887587, 0.06716646, 0.02..."
4,0,factsguide: society now #motivation,factsguide society now motivation,"[-0.016824858, -0.011649432, 0.100801155, -0.0..."
...,...,...,...,...
94,0,omg!!! loving this station!!! way to jam out a...,omg loving this station way to jam out at work...,"[-0.011202576, -0.030689226, 0.08444457, 0.012..."
95,0,@user i'll always hope that one day i'll get t...,USER i'll always hope that one day i'll get to...,"[0.004627814, -0.003055276, 0.09976651, 0.0032..."
96,0,#model i love u take with u all the time in ur...,model i love u take with u all the time in urã...,"[-0.0042502587, -0.044887587, 0.06716646, 0.02..."
97,0,couple having sex fat naked japanese girls,couple having sex fat naked japanese girls,"[-0.0075420453, -0.032993905, 0.0986156, 0.018..."


In [ ]:
def prepare_data(pickle_filepath):
  data = pickle.load(open(pickle_filepath,'rb'))
  
  X = np.array(list(data.sent_vec.apply(lambda x: list(x)).values))
  
  return X, data.label.values

In [ ]:
Xtest, ytest = prepare_data("twitter-test-data-new.pkl")

In [ ]:
Xtest.shape, ytest.shape

((99, 300), (99,))

In [ ]:
del fasttext_emb

## Test models

In [ ]:
!gdown https://drive.google.com/file/d/16RTmAYGJ-FWRJ1R_oYcNE-3opdopb37X/view --fuzzy

Downloading...
From: https://drive.google.com/uc?id=16RTmAYGJ-FWRJ1R_oYcNE-3opdopb37X
To: /content/models.rar
100% 116M/116M [00:01<00:00, 72.0MB/s]


In [ ]:
!mkdir models
!unrar e models.rar /content/models


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from models.rar

Extracting  /content/models/finaldata_random_forest.joblib                 3%  7% 10% 14% 18% 21% 25% 28% 32% 36% 39% 43% 46% 50% 54%  OK 
Extracting  /content/models/finaldata_random_forest_undersampled.joblib      54% 55% 56% 57%  OK 
Extracting  /content/models/finaldata_xgboost_model.json                  60% 63%  OK 
Extracting  /content/models/finaldata_xgboost_model_undersampled.json      67% 70% 71%  OK 
Extracting  /content/models/finaldata_xgboost_model_undersampled.joblib      75% 78% 79%  OK 
Extracting  /content/models/finaldata_xgboost_model.pkl                   83% 85%  OK 
Extracting  /content/models/finaldata_xgboost_model.joblib                89% 91%  OK 
Extracting  /content/models/finaldata_xgboost_model_undersam

In [ ]:
import joblib
from sklearn.metrics import classification_report

### Random Forest

In [ ]:
rf = joblib.load("/content/models/finaldata_random_forest.joblib")

In [ ]:
predictions = rf.predict(Xtest)
# print(classification_report(ytest, predictions)) #Random forest

ValueError: ignored

In [ ]:
testdf = pd.DataFrame()

In [ ]:
testdf['Random Forest'] = predictions

### Random Forest - Undersampled

In [ ]:
rf = joblib.load("/content/models/finaldata_random_forest_undersampled.joblib")

In [ ]:
predictions = rf.predict(Xtest)
# print(classification_report(ytest, predictions)) #Random forest

ValueError: ignored

In [ ]:
testdf['Random Forest - Undersampled'] = predictions

### XGBoost

In [ ]:
xgb_model = joblib.load("/content/models/finaldata_xgboost_model.joblib")

In [ ]:
predictions = xgb_model.predict(Xtest)
# print(classification_report(ytest, predictions)) #xgboost

In [ ]:
testdf['XGBoost'] = predictions

### XGBoost - Undersampled

In [ ]:
xgb_model = joblib.load("/content/models/finaldata_xgboost_model_undersampled.joblib")

In [ ]:
predictions = xgb_model.predict(Xtest)
# print(classification_report(ytest, predictions)) #xgboost

In [ ]:
testdf['XGBoost - Undersampled'] = predictions

In [ ]:
testdf

,Random Forest,Random Forest - Undersampled,XGBoost,XGBoost - Undersampled
0,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE
1,NEUTRAL,NEUTRAL,NEGATIVE,NEGATIVE
2,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE
3,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE
4,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
...,...,...,...,...
94,POSITIVE,POSITIVE,POSITIVE,POSITIVE
95,NEUTRAL,NEGATIVE,POSITIVE,POSITIVE
96,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE
97,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE


### BERTweet

In [ ]:
data = pickle.load(open("/content/twitter-test-data-new.pkl", "rb"))
data

,label,tweet,cleaned_tweets,sent_vec
0,0,@user when a father is dysfunctional and is so...,USER when a father is dysfunctional and is so ...,"[-0.014220357, -0.008476666, 0.08933757, 0.003..."
1,0,@user @user thanks for #lyft credit i can't us...,USER USER thanks for lyft credit i cannot use ...,"[-0.0136345625, -0.04320838, 0.083086476, -0.0..."
2,0,bihday your majesty,bihday your majesty,"[-0.024211615, -0.04941867, 0.0942983, -0.0160..."
3,0,#model i love u take with u all the time in ur...,model i love u take with u all the time in urã...,"[-0.0042502587, -0.044887587, 0.06716646, 0.02..."
4,0,factsguide: society now #motivation,factsguide society now motivation,"[-0.016824858, -0.011649432, 0.100801155, -0.0..."
...,...,...,...,...
94,0,omg!!! loving this station!!! way to jam out a...,omg loving this station way to jam out at work...,"[-0.011202576, -0.030689226, 0.08444457, 0.012..."
95,0,@user i'll always hope that one day i'll get t...,USER i'll always hope that one day i'll get to...,"[0.004627814, -0.003055276, 0.09976651, 0.0032..."
96,0,#model i love u take with u all the time in ur...,model i love u take with u all the time in urã...,"[-0.0042502587, -0.044887587, 0.06716646, 0.02..."
97,0,couple having sex fat naked japanese girls,couple having sex fat naked japanese girls,"[-0.0075420453, -0.032993905, 0.0986156, 0.018..."


In [ ]:
testdf['BERTweet'] = data['cleaned_tweets'].apply(sa1.inference)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# print(classification_report(ytest, testdf['BERTweet'].values)) #BERtweet

              precision    recall  f1-score   support

    NEGATIVE       0.50      0.60      0.55        20
     NEUTRAL       0.21      0.21      0.21        14
    POSITIVE       0.83      0.71      0.77        28

    accuracy                           0.56        62
   macro avg       0.52      0.51      0.51        62
weighted avg       0.59      0.56      0.57        62



### BERT base

In [ ]:
testdf['bert-base-multilingual-uncased-sentiment_clean'] = data['cleaned_tweets'].apply(sa2.inference)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# print(classification_report(ytest, testdf['bert-base-multilingual-uncased-sentiment_clean'].values)) #BERT Base

              precision    recall  f1-score   support

    NEGATIVE       0.52      0.70      0.60        20
     NEUTRAL       1.00      0.07      0.13        14
    POSITIVE       0.71      0.86      0.77        28

    accuracy                           0.63        62
   macro avg       0.74      0.54      0.50        62
weighted avg       0.71      0.63      0.57        62



In [ ]:
testdf['tweet'] = data['tweet']
testdf['label'] = data['label']

testdf['cleaned_tweets'] = data['cleaned_tweets']

testdf

,Random Forest,Random Forest - Undersampled,XGBoost,XGBoost - Undersampled,BERTweet,bert-base-multilingual-uncased-sentiment_clean,tweet,label,cleaned_tweets
0,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,@user when a father is dysfunctional and is so...,0,USER when a father is dysfunctional and is so ...
1,NEUTRAL,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,@user @user thanks for #lyft credit i can't us...,0,USER USER thanks for lyft credit i cannot use ...
2,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,POSITIVE,bihday your majesty,0,bihday your majesty
3,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,#model i love u take with u all the time in ur...,0,model i love u take with u all the time in urã...
4,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE,factsguide: society now #motivation,0,factsguide society now motivation
...,...,...,...,...,...,...,...,...,...
94,POSITIVE,POSITIVE,POSITIVE,POSITIVE,POSITIVE,POSITIVE,omg!!! loving this station!!! way to jam out a...,0,omg loving this station way to jam out at work...
95,NEUTRAL,NEGATIVE,POSITIVE,POSITIVE,NEUTRAL,NEUTRAL,@user i'll always hope that one day i'll get t...,0,USER i'll always hope that one day i'll get to...
96,NEUTRAL,NEUTRAL,NEUTRAL,POSITIVE,POSITIVE,POSITIVE,#model i love u take with u all the time in ur...,0,model i love u take with u all the time in urã...
97,NEUTRAL,NEGATIVE,NEGATIVE,NEGATIVE,NEUTRAL,NEGATIVE,couple having sex fat naked japanese girls,0,couple having sex fat naked japanese girls


In [ ]:
testdf.to_excel("twitter-sample-data-results-new.xlsx")

In [ ]:
with open("./test-predictions-undersampled.pkl","wb") as f:
  pickle.dump(testdf, f)